This notebook acts to create Elastic Net models on various versions of the Datasets  GSE40279, which was used by Hannum et. al. for their elastic net models (in https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3780611/), with certain portions of the non-clock CpGs removed. We first train an Elatic Net model on the Original Dataset and find the CpGs with non zero weights, Clock CpGs, which the model produces. We then create a table of CpGs which are not Clock CpGs in the original model. Then we create a new dataset by removing an integer percent of the non-clock CpGs. We remove the CpGs randomly and independently, such that for each new dataset a selection of the whole non-Clock CpG dataset is chosen. CpGs are chosen using the random library. Then we train an Elastic Net model for each dataset. New Datasets and EN are created with 1-10% of non-Clock CpGs removed, in 1% intervals, and with 10-100% removed, in 10% intervals.

In [1]:
#import the necessary Libraries for this program
import pandas as pd 
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import ElasticNetCV
import geo_tools as gt 
from joblib import dump, load
from sklearn.preprocessing import StandardScaler
random

%matplotlib inline

In [2]:
#find the parent directory
import os
import sys
sys.path.append('..')

print(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
parent = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

/Users/connorluellen/Desktop/USB/clocks


In [2]:
#When given two lists as inputs this function finds all of the common items between the two lists and returns
#the results as a list.
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

# Generate a dataset for GSE40279
Here we use the custom library geo_tools which reads in the table from the downloaded table. You must download and then store the desired GSE in the data file in a folder with it's name

In [3]:
#Read in the table with Geo Tools
hannum_raw =  gt.__matrix_to_df("data/GSE40279/GSE40279_series_matrix.txt", GSE = "GSE40279")

In [4]:
hannum_raw

,GSM989827,GSM989828,GSM989829,GSM989830,GSM989831,GSM989832,GSM989833,GSM989834,GSM989835,GSM989836,...,GSM990618,GSM990619,GSM990620,GSM990621,GSM990622,GSM990623,GSM990624,GSM990625,GSM990626,GSM990627
ID_REF,,,,,,,,,,,,,,,,,,,,,
cg00000029,0.464197,0.454883,0.485764,0.480785,0.501219,0.499918,0.485852,0.512442,0.518155,0.417986,...,0.560958,0.472081,0.508502,0.505193,0.443411,0.527496,0.588331,0.362994,0.499145,0.458600
cg00000108,0.941091,0.939033,0.918802,0.929908,0.934548,0.950543,0.925855,0.941330,0.938528,0.933814,...,0.934699,0.978612,0.922024,0.963052,0.992631,0.958173,0.982450,0.954392,0.931691,0.974731
cg00000109,0.911182,0.596455,0.870333,0.889689,0.890450,0.898493,0.893972,0.892010,0.900841,0.883539,...,0.881957,0.926289,0.930091,0.946547,0.929131,0.922034,0.855145,0.927183,0.900938,0.829869
cg00000165,0.132014,0.206917,0.162861,0.197780,0.148437,0.224093,0.400489,0.194553,0.134710,0.204569,...,0.199883,0.165116,0.210248,0.177351,0.118742,0.223068,0.162180,0.196430,0.167477,0.170578
cg00000236,0.717861,0.723935,0.719196,0.704061,0.754913,0.829192,0.723782,0.695142,0.731872,0.742913,...,0.759011,0.792883,0.730367,0.783830,0.787089,0.778959,0.796869,0.713020,0.730215,0.782844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ch.9.98937537R,0.042808,0.036811,0.042844,0.042258,0.039613,0.035309,0.031304,0.031119,0.031373,0.039919,...,0.051150,0.049286,0.071773,0.056795,0.041892,0.049571,0.055904,0.048526,0.056429,0.040701
ch.9.98957343R,0.052589,0.053343,0.045973,0.048733,0.039254,0.043023,0.037075,0.048277,0.041296,0.049226,...,0.051119,0.058118,0.059010,0.057837,0.037403,0.033060,0.062442,0.041087,0.047835,0.027499
ch.9.98959675F,0.035624,0.075618,0.126421,0.084051,0.165874,0.088889,0.097599,0.084294,0.052505,0.098726,...,0.077570,0.087988,0.098583,0.066481,0.001334,0.093414,0.101100,0.049857,0.028896,0.000000


In [5]:
#Save the dataset
hannum_raw.to_pickle(parent + '/MethylAndAges/Hannum raw.pkl')

In [26]:
age = []
for item in hannum_raw.columns.to_list():
    
    age.append(gt.info("GSE40279", item))

In [27]:
ages = pd.DataFrame(data=age, index=hannum_raw.columns.to_list())
ages

,0
GSM989827,67.0
GSM989828,89.0
GSM989829,66.0
GSM989830,64.0
GSM989831,62.0
...,...
GSM990623,78.0
GSM990624,71.0
GSM990625,68.0
GSM990626,61.0


In [ ]:
ages.to_pickle(parent + '/MethylAndAges/Hannum ages.pkl')


In [7]:
#Transpose our data such that each row is a GSM. This is needed to allow Standardization with StandardScaler
hannum_test = hannum_raw.T
hannum_test.columns.name = "CpG"
hannum_test 

CpG,cg00000029,cg00000108,cg00000109,cg00000165,cg00000236,cg00000289,cg00000292,cg00000321,cg00000363,cg00000622,...,ch.9.945770F,ch.9.96055087R,ch.9.97139671F,ch.9.98463211R,ch.9.98936572R,ch.9.98937537R,ch.9.98957343R,ch.9.98959675F,ch.9.98989607R,ch.9.991104F
GSM989827,0.464197,0.941091,0.911182,0.132014,0.717861,0.686521,0.805003,0.228244,0.338483,0.016508,...,0.022659,0.109918,0.061222,0.034284,0.133692,0.042808,0.052589,0.035624,0.028066,0.043850
GSM989828,0.454883,0.939033,0.596455,0.206917,0.723935,0.619084,0.814672,0.310879,0.418998,0.005747,...,0.005095,0.076996,0.052640,0.027978,0.125270,0.036811,0.053343,0.075618,0.017428,0.032950
GSM989829,0.485764,0.918802,0.870333,0.162861,0.719196,0.635678,0.824336,0.263215,0.424736,0.012197,...,0.021444,0.070694,0.058888,0.032643,0.139105,0.042844,0.045973,0.126421,0.021752,0.022375
GSM989830,0.480785,0.929908,0.889689,0.197780,0.704061,0.610864,0.811152,0.316761,0.398772,0.019945,...,0.028587,0.094749,0.056279,0.036997,0.140601,0.042258,0.048733,0.084051,0.027504,0.053007
GSM989831,0.501219,0.934548,0.890450,0.148437,0.754913,0.651262,0.808628,0.338289,0.366965,0.000000,...,0.018626,0.110543,0.057568,0.036746,0.129993,0.039613,0.039254,0.165874,0.020889,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM990623,0.527496,0.958173,0.922034,0.223068,0.778959,0.709248,0.825768,0.354296,0.396241,0.001778,...,0.016319,0.079741,0.072076,0.052052,0.159212,0.049571,0.033060,0.093414,0.013654,0.014088
GSM990624,0.588331,0.982450,0.855145,0.162180,0.796869,0.535831,0.806713,0.295598,0.301319,0.006685,...,0.023837,0.117762,0.058466,0.037199,0.133899,0.055904,0.062442,0.101100,0.014193,0.000000
GSM990625,0.362994,0.954392,0.927183,0.196430,0.713020,0.664184,0.804045,0.395724,0.445179,0.003822,...,0.012054,0.079249,0.053499,0.050002,0.149589,0.048526,0.041087,0.049857,0.016840,0.041415
GSM990626,0.499145,0.931691,0.900938,0.167477,0.730215,0.665792,0.831365,0.338117,0.383953,0.000000,...,0.009878,0.090751,0.060335,0.040898,0.168269,0.056429,0.047835,0.028896,0.025346,0.052959


In [8]:
#Split the dataset into training and test subsets 
from sklearn.model_selection import train_test_split
methyl_raw_train, methyl_raw_test, age_train, age_test = train_test_split(hannum_test, ages, test_size=0.2, random_state=42)


In [9]:
#Scale our data such that the fit is to the training set
scaler = StandardScaler().fit(methyl_raw_train)
methyl_train = scaler.transform(methyl_raw_train)

methyl_test = scaler.transform(methyl_raw_test)

In [10]:
#Read in age training data as a list
Y_train=age_train.values.ravel()

In [11]:
#Create Elastic Net model
elastic_netCV_original = ElasticNetCV(l1_ratio = 0.5, n_alphas = 50, cv = 10, n_jobs=11, random_state = 42,
                             max_iter=5000, tol = 0.001, selection='cyclic')

In [12]:
#Train the model. 
elastic_netCV_original.fit(methyl_train,Y_train)

ElasticNetCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
             l1_ratio=0.5, max_iter=5000, n_alphas=50, n_jobs=7,
             normalize=False, positive=False, precompute='auto',
             random_state=42, selection='cyclic', tol=0.001, verbose=0)

In [13]:
#Save the Elastic Net model
dump(elastic_netCV_original, parent + '/Trained_Models/elastic_netCV_Hannum_original.joblib')

['elastic_netCV_Hannum_original.joblib']

In [15]:
#Get the non-zero coefficients to get the significant cpgs.
coeffs_original = pd.DataFrame(elastic_netCV_original.coef_)
coeffs_original = coeffs_original[(coeffs_original.T != 0).any()]
coeffs_original = coeffs_original.rename(columns={coeffs_original.columns[0]: 'Magnitude'})
coeffs_original

,Magnitude
1029,0.044580
1121,-0.015875
1303,-0.010844
1363,0.050709
2722,0.009543
...,...
470604,-0.132771
471369,0.016129
472204,0.026252
472489,-0.029873


In [16]:
#Get significant CpGs and their indices
colnames = pd.DataFrame(hannum_test.columns)
sig_cpgs_original = colnames.iloc[coeffs_original.index]
sig_cpgs_original

,CpG
1029,cg00042478
1121,cg00047050
1303,cg00055986
1363,cg00058879
2722,cg00120464
...,...
470604,ch.11.315572F
471369,ch.19.1716179R
472204,ch.4.60646130F
472489,ch.6.2574971F


# Now Let us Remove  Some non significant CpGs. Let us find the list of all Non-clock CpGs

In [20]:
# Create a list of all non clock CpGs
first = True
for cpg in sig_cpgs_original["CpG"].to_list():
    
    if first == True:
        nonsig_cpgs_original = colnames[colnames["CpG"].str.contains(cpg)==False]
        first = False
    else:
        nonsig_cpgs_original = nonsig_cpgs_original[nonsig_cpgs_original["CpG"].str.contains(cpg)==False]
        
nonsig_cpgs_original

,CpG
"(cg00000029,)",cg00000029
"(cg00000108,)",cg00000108
"(cg00000109,)",cg00000109
"(cg00000165,)",cg00000165
"(cg00000236,)",cg00000236
...,...
"(ch.9.98937537R,)",ch.9.98937537R
"(ch.9.98957343R,)",ch.9.98957343R
"(ch.9.98959675F,)",ch.9.98959675F
"(ch.9.98989607R,)",ch.9.98989607R


In [21]:
#Find length of non clock CpGs 
len_nonsig_original = len(nonsig_cpgs_original)

# Now remove 1% CpGs
Here we will remove 1% of nonsig CpGs, create the dataset, and train an EN model

In [23]:
#Find number to be removed
num_removed = int(len_nonsig_original*0.01)

In [24]:
#Generate CpGs to be removed
list_removed = random.sample(range(0, len_nonsig_original), num_removed)

cpgs_removed_1 = nonsig_cpgs_original.iloc[list_removed]
cpgs_removed_1

,CpG
"(cg12194727,)",cg12194727
"(cg21499430,)",cg21499430
"(cg21229954,)",cg21229954
"(cg01030598,)",cg01030598
"(cg20344434,)",cg20344434
...,...
"(cg15230883,)",cg15230883
"(cg13661323,)",cg13661323
"(cg02310851,)",cg02310851
"(cg09793796,)",cg09793796


In [25]:
#Generate a new data set with the randomly selected CpGs removed
hannum_test_1 = hannum_test.drop(cpgs_removed_1["CpG"].to_list(), axis=1)

hannum_test_1

CpG,cg00000029,cg00000108,cg00000109,cg00000165,cg00000236,cg00000289,cg00000292,cg00000321,cg00000363,cg00000622,...,ch.9.945770F,ch.9.96055087R,ch.9.97139671F,ch.9.98463211R,ch.9.98936572R,ch.9.98937537R,ch.9.98957343R,ch.9.98959675F,ch.9.98989607R,ch.9.991104F
GSM989827,0.464197,0.941091,0.911182,0.132014,0.717861,0.686521,0.805003,0.228244,0.338483,0.016508,...,0.022659,0.109918,0.061222,0.034284,0.133692,0.042808,0.052589,0.035624,0.028066,0.043850
GSM989828,0.454883,0.939033,0.596455,0.206917,0.723935,0.619084,0.814672,0.310879,0.418998,0.005747,...,0.005095,0.076996,0.052640,0.027978,0.125270,0.036811,0.053343,0.075618,0.017428,0.032950
GSM989829,0.485764,0.918802,0.870333,0.162861,0.719196,0.635678,0.824336,0.263215,0.424736,0.012197,...,0.021444,0.070694,0.058888,0.032643,0.139105,0.042844,0.045973,0.126421,0.021752,0.022375
GSM989830,0.480785,0.929908,0.889689,0.197780,0.704061,0.610864,0.811152,0.316761,0.398772,0.019945,...,0.028587,0.094749,0.056279,0.036997,0.140601,0.042258,0.048733,0.084051,0.027504,0.053007
GSM989831,0.501219,0.934548,0.890450,0.148437,0.754913,0.651262,0.808628,0.338289,0.366965,0.000000,...,0.018626,0.110543,0.057568,0.036746,0.129993,0.039613,0.039254,0.165874,0.020889,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM990623,0.527496,0.958173,0.922034,0.223068,0.778959,0.709248,0.825768,0.354296,0.396241,0.001778,...,0.016319,0.079741,0.072076,0.052052,0.159212,0.049571,0.033060,0.093414,0.013654,0.014088
GSM990624,0.588331,0.982450,0.855145,0.162180,0.796869,0.535831,0.806713,0.295598,0.301319,0.006685,...,0.023837,0.117762,0.058466,0.037199,0.133899,0.055904,0.062442,0.101100,0.014193,0.000000
GSM990625,0.362994,0.954392,0.927183,0.196430,0.713020,0.664184,0.804045,0.395724,0.445179,0.003822,...,0.012054,0.079249,0.053499,0.050002,0.149589,0.048526,0.041087,0.049857,0.016840,0.041415
GSM990626,0.499145,0.931691,0.900938,0.167477,0.730215,0.665792,0.831365,0.338117,0.383953,0.000000,...,0.009878,0.090751,0.060335,0.040898,0.168269,0.056429,0.047835,0.028896,0.025346,0.052959


In [26]:
#ensure that all originalclock CpGs remain in the new set
common = intersection(hannum_test_1.columns.to_list(), sig_cpgs_original["CpG"].to_list())

if not sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    raise ValueError('Some Significant CpGs lost!')

In [27]:
#Split the dataset into training and test subsets 

methyl_raw_train, methyl_raw_test, age_train, age_test = train_test_split(hannum_test_1, ages, test_size=0.2, random_state=42)


In [28]:
#Read in age training data as a list
Y_train=age_train.values.ravel()

In [29]:
#Scale our data such that the fit is to the training set
scaler = StandardScaler().fit(methyl_raw_train)
methyl_train = scaler.transform(methyl_raw_train)

methyl_test = scaler.transform(methyl_raw_test)

In [30]:
#Create Elastic Net model
elastic_netCV_1 = ElasticNetCV(l1_ratio = 0.5, n_alphas = 50, cv = 10, n_jobs=11, random_state = 42,
                             max_iter=5000, tol = 0.001, selection='cyclic')

In [31]:
#Train the model. 
elastic_netCV_1.fit(methyl_train,Y_train)

ElasticNetCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
             l1_ratio=0.5, max_iter=5000, n_alphas=50, n_jobs=11,
             normalize=False, positive=False, precompute='auto',
             random_state=42, selection='cyclic', tol=0.001, verbose=0)

In [32]:
#Save the Elastic Net model
dump(elastic_netCV_1, parent + '/Trained_Models/elastic_netCV_Hannum_1_i5000.joblib')

['D:\\elastic_netCV_Hannum_1_i5000.joblib']

In [33]:
#Save the Dataset
hannum_test_1.to_pickle(parent + '/MethylAndAges/hannum_1% nonsig removed.pkl')

# Find significant CpGs and compare
Here, as an assurance that our trainings are working, we will find the new Clock CpGs and compare them to the original

In [34]:
#Get the non-zero coefficients to get the significant cpgs. 
coeffs_1 = pd.DataFrame(elastic_netCV_1.coef_)
coeffs_1 = coeffs_1[(coeffs_1.T != 0).any()]
coeffs_1

,0
1014,0.048104
1102,-0.022901
1283,-0.010881
1342,0.051129
2687,0.012739
...,...
465915,-0.136172
466672,0.027560
467496,0.032633
467778,-0.034797


In [35]:
#Get significant CpGs and their indices
colnames_1 = pd.DataFrame(hannum_test_1.columns)
sig_cpgs_1 = colnames_1.iloc[coeffs_1.index]
sig_cpgs_1

,CpG
1014,cg00042478
1102,cg00047050
1283,cg00055986
1342,cg00058879
2687,cg00120464
...,...
465915,ch.11.315572F
466672,ch.19.1716179R
467496,ch.4.60646130F
467778,ch.6.2574971F


In [36]:
#Compare original and new Clock CpGs
common = intersection(sig_cpgs_1["CpG"].to_list(), sig_cpgs_original["CpG"].to_list())


if  sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    print('NO Significant CpGs lost!')

In [37]:
len(common)

830

# Now remove 2% CpGs
Here we will remove 2% of nonsig CpGs, create the dataset, and train an EN model

In [38]:
#Find the number to be removed
num_removed = int(len_nonsig_original*0.02)

In [39]:
#Generate CpGs to be removed
list_removed = random.sample(range(0, len_nonsig_original), num_removed)

cpgs_removed_2 = nonsig_cpgs_original.iloc[list_removed]
cpgs_removed_2

,CpG
"(cg13045637,)",cg13045637
"(cg14697880,)",cg14697880
"(cg26378201,)",cg26378201
"(cg12055422,)",cg12055422
"(cg00540593,)",cg00540593
...,...
"(cg10908625,)",cg10908625
"(cg00099306,)",cg00099306
"(cg25745663,)",cg25745663
"(cg25412384,)",cg25412384


In [40]:
#Generate a new data set with the randomly selected CpGs removed
hannum_test_2 = hannum_test.drop(cpgs_removed_2["CpG"].to_list(), axis=1)

hannum_test_2

CpG,cg00000029,cg00000108,cg00000109,cg00000165,cg00000236,cg00000289,cg00000292,cg00000321,cg00000363,cg00000622,...,ch.9.945770F,ch.9.96055087R,ch.9.97139671F,ch.9.98463211R,ch.9.98936572R,ch.9.98937537R,ch.9.98957343R,ch.9.98959675F,ch.9.98989607R,ch.9.991104F
GSM989827,0.464197,0.941091,0.911182,0.132014,0.717861,0.686521,0.805003,0.228244,0.338483,0.016508,...,0.022659,0.109918,0.061222,0.034284,0.133692,0.042808,0.052589,0.035624,0.028066,0.043850
GSM989828,0.454883,0.939033,0.596455,0.206917,0.723935,0.619084,0.814672,0.310879,0.418998,0.005747,...,0.005095,0.076996,0.052640,0.027978,0.125270,0.036811,0.053343,0.075618,0.017428,0.032950
GSM989829,0.485764,0.918802,0.870333,0.162861,0.719196,0.635678,0.824336,0.263215,0.424736,0.012197,...,0.021444,0.070694,0.058888,0.032643,0.139105,0.042844,0.045973,0.126421,0.021752,0.022375
GSM989830,0.480785,0.929908,0.889689,0.197780,0.704061,0.610864,0.811152,0.316761,0.398772,0.019945,...,0.028587,0.094749,0.056279,0.036997,0.140601,0.042258,0.048733,0.084051,0.027504,0.053007
GSM989831,0.501219,0.934548,0.890450,0.148437,0.754913,0.651262,0.808628,0.338289,0.366965,0.000000,...,0.018626,0.110543,0.057568,0.036746,0.129993,0.039613,0.039254,0.165874,0.020889,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM990623,0.527496,0.958173,0.922034,0.223068,0.778959,0.709248,0.825768,0.354296,0.396241,0.001778,...,0.016319,0.079741,0.072076,0.052052,0.159212,0.049571,0.033060,0.093414,0.013654,0.014088
GSM990624,0.588331,0.982450,0.855145,0.162180,0.796869,0.535831,0.806713,0.295598,0.301319,0.006685,...,0.023837,0.117762,0.058466,0.037199,0.133899,0.055904,0.062442,0.101100,0.014193,0.000000
GSM990625,0.362994,0.954392,0.927183,0.196430,0.713020,0.664184,0.804045,0.395724,0.445179,0.003822,...,0.012054,0.079249,0.053499,0.050002,0.149589,0.048526,0.041087,0.049857,0.016840,0.041415
GSM990626,0.499145,0.931691,0.900938,0.167477,0.730215,0.665792,0.831365,0.338117,0.383953,0.000000,...,0.009878,0.090751,0.060335,0.040898,0.168269,0.056429,0.047835,0.028896,0.025346,0.052959


In [41]:
#ensure that all clock CpGs remain in the new set
common = intersection(hannum_test_2.columns.to_list(), sig_cpgs_original["CpG"].to_list())

if not sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    raise ValueError('Some Significant CpGs lost!')

In [42]:
len(common)

847

In [43]:
#Split the dataset into training and test subsets 

methyl_raw_train, methyl_raw_test, age_train, age_test = train_test_split(hannum_test_2, ages, test_size=0.2, random_state=42)


In [44]:
#Scale our data such that the fit is to the training set
scaler = StandardScaler().fit(methyl_raw_train)
methyl_train = scaler.transform(methyl_raw_train)

methyl_test = scaler.transform(methyl_raw_test)

In [45]:
#Read in age training data as a list
Y_train=age_train.values.ravel()

In [46]:
#Create Elastic Net model
elastic_netCV_2_i5000 = ElasticNetCV(l1_ratio = 0.5, n_alphas = 50, cv = 10, n_jobs=4, random_state = 42,
                             max_iter=5000, tol = 0.001, selection='cyclic')

In [47]:
#Train the model. 
elastic_netCV_2_i5000.fit(methyl_train,Y_train)

ElasticNetCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
             l1_ratio=0.5, max_iter=5000, n_alphas=50, n_jobs=4,
             normalize=False, positive=False, precompute='auto',
             random_state=42, selection='cyclic', tol=0.001, verbose=0)

In [48]:
#Save the Elastic Net model
dump(elastic_netCV_2_i5000, parent + '/Trained_Models/elastic_netCV_Hannum_2_i5000.joblib')

['D:\\elastic_netCV_Hannum_2_i5000.joblib']

In [49]:
#Save the Dataset
hannum_test_2.to_pickle(parent + '/MethylAndAges/hannum_2% nonsig removed.pkl')

# Find significant CpGs and compare
Here, as an assurance that our trainings are working, we will find the new Clock CpGs and compare them to the original

In [50]:
#Get the non-zero coefficients to get the significant cpgs. Runtime ~3 min
coeffs_2_i5000 = pd.DataFrame(elastic_netCV_2_i5000.coef_)
coeffs_2_i5000 = coeffs_2_i5000[(coeffs_2_i5000.T != 0).any()]
coeffs_2_i5000

,0
1015,0.049978
1107,-0.025640
1287,-0.013315
1346,0.049547
2668,0.013426
...,...
461962,0.030398
462778,0.035565
462938,-0.000014
463058,-0.037691


In [51]:
#Get significant CpGs and their indices
colnames_2_i5000 = pd.DataFrame(hannum_test_2.columns)
sig_cpgs_2_i5000 = colnames_2_i5000.iloc[coeffs_2_i5000.index]
sig_cpgs_2_i5000

,CpG
1015,cg00042478
1107,cg00047050
1287,cg00055986
1346,cg00058879
2668,cg00120464
...,...
461962,ch.19.1716179R
462778,ch.4.60646130F
462938,ch.5.67026991F
463058,ch.6.2574971F


In [52]:
#Compare original and new Clock CpGs
common = intersection(sig_cpgs_2_i5000["CpG"].to_list(), sig_cpgs_original["CpG"].to_list())


if  sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    print('NO Significant CpGs lost!')

In [53]:
len(common)

827

# Now remove 3% CpGs
Here we will remove 3% of nonsig CpGs, create the dataset, and train an EN model

In [54]:
#Find number to be removed
num_removed = int(len_nonsig_original*0.03)

In [55]:
#Generate CpGs to be removed
list_removed = random.sample(range(0, len_nonsig_original), num_removed)

cpgs_removed_3 = nonsig_cpgs_original.iloc[list_removed]
cpgs_removed_3

,CpG
"(cg17292337,)",cg17292337
"(cg18456942,)",cg18456942
"(cg26712080,)",cg26712080
"(cg03131366,)",cg03131366
"(cg08107079,)",cg08107079
...,...
"(cg19269201,)",cg19269201
"(cg24990317,)",cg24990317
"(cg01439754,)",cg01439754
"(cg04993112,)",cg04993112


In [56]:
#Generate a new data set with the randomly selected CpGs removed
hannum_test_3 = hannum_test.drop(cpgs_removed_3["CpG"].to_list(), axis=1)

hannum_test_3

CpG,cg00000029,cg00000108,cg00000109,cg00000165,cg00000236,cg00000289,cg00000292,cg00000321,cg00000363,cg00000622,...,ch.9.945770F,ch.9.96055087R,ch.9.97139671F,ch.9.98463211R,ch.9.98936572R,ch.9.98937537R,ch.9.98957343R,ch.9.98959675F,ch.9.98989607R,ch.9.991104F
GSM989827,0.464197,0.941091,0.911182,0.132014,0.717861,0.686521,0.805003,0.228244,0.338483,0.016508,...,0.022659,0.109918,0.061222,0.034284,0.133692,0.042808,0.052589,0.035624,0.028066,0.043850
GSM989828,0.454883,0.939033,0.596455,0.206917,0.723935,0.619084,0.814672,0.310879,0.418998,0.005747,...,0.005095,0.076996,0.052640,0.027978,0.125270,0.036811,0.053343,0.075618,0.017428,0.032950
GSM989829,0.485764,0.918802,0.870333,0.162861,0.719196,0.635678,0.824336,0.263215,0.424736,0.012197,...,0.021444,0.070694,0.058888,0.032643,0.139105,0.042844,0.045973,0.126421,0.021752,0.022375
GSM989830,0.480785,0.929908,0.889689,0.197780,0.704061,0.610864,0.811152,0.316761,0.398772,0.019945,...,0.028587,0.094749,0.056279,0.036997,0.140601,0.042258,0.048733,0.084051,0.027504,0.053007
GSM989831,0.501219,0.934548,0.890450,0.148437,0.754913,0.651262,0.808628,0.338289,0.366965,0.000000,...,0.018626,0.110543,0.057568,0.036746,0.129993,0.039613,0.039254,0.165874,0.020889,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM990623,0.527496,0.958173,0.922034,0.223068,0.778959,0.709248,0.825768,0.354296,0.396241,0.001778,...,0.016319,0.079741,0.072076,0.052052,0.159212,0.049571,0.033060,0.093414,0.013654,0.014088
GSM990624,0.588331,0.982450,0.855145,0.162180,0.796869,0.535831,0.806713,0.295598,0.301319,0.006685,...,0.023837,0.117762,0.058466,0.037199,0.133899,0.055904,0.062442,0.101100,0.014193,0.000000
GSM990625,0.362994,0.954392,0.927183,0.196430,0.713020,0.664184,0.804045,0.395724,0.445179,0.003822,...,0.012054,0.079249,0.053499,0.050002,0.149589,0.048526,0.041087,0.049857,0.016840,0.041415
GSM990626,0.499145,0.931691,0.900938,0.167477,0.730215,0.665792,0.831365,0.338117,0.383953,0.000000,...,0.009878,0.090751,0.060335,0.040898,0.168269,0.056429,0.047835,0.028896,0.025346,0.052959


In [57]:
#ensure that all clock CpGs remain in the new set
common = intersection(hannum_test_3.columns.to_list(), sig_cpgs_original["CpG"].to_list())

if not sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    raise ValueError('Some Significant CpGs lost!')

In [58]:
len(common)

847

In [59]:
#Split the dataset into training and test subsets 

methyl_raw_train, methyl_raw_test, age_train, age_test = train_test_split(hannum_test_3, ages, test_size=0.2, random_state=42)


In [60]:
#Scale our data such that the fit is to the training set
scaler = StandardScaler().fit(methyl_raw_train)
methyl_train = scaler.transform(methyl_raw_train)

methyl_test = scaler.transform(methyl_raw_test)

In [61]:
#Read in age training data as a list
Y_train=age_train.values.ravel()

In [62]:
#Create Elastic Net model
elastic_netCV_3_i5000 = ElasticNetCV(l1_ratio = 0.5, n_alphas = 50, cv = 10, n_jobs=3, random_state = 42,
                             max_iter=5000, tol = 0.001, selection='cyclic')

In [63]:
#Train the model. 
elastic_netCV_3_i5000.fit(methyl_train,Y_train)

ElasticNetCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
             l1_ratio=0.5, max_iter=5000, n_alphas=50, n_jobs=3,
             normalize=False, positive=False, precompute='auto',
             random_state=42, selection='cyclic', tol=0.001, verbose=0)

In [64]:
#Save the Elastic Net model
dump(elastic_netCV_3_i5000, parent + '/Trained_Models/elastic_netCV_Hannum_3_i5000.joblib')

['elastic_netCV_Hannum_3_i5000.joblib']

In [65]:
#Save the Dataset
hannum_test_3.to_pickle(parent + '/MethylAndAges/hannum_3% nonsig removed.pkl')

# Now remove 4% CpGs
Here we will remove 4% of nonsig CpGs, create the dataset, and train an EN model

In [66]:
#Find number to be removed
num_removed = int(len_nonsig_original*0.04)

In [67]:
#Generate CpGs to be removed
list_removed = random.sample(range(0, len_nonsig_original), num_removed)

cpgs_removed_4 = nonsig_cpgs_original.iloc[list_removed]
cpgs_removed_4

,CpG
"(cg08580838,)",cg08580838
"(cg04673912,)",cg04673912
"(cg16361947,)",cg16361947
"(cg26151597,)",cg26151597
"(cg23375068,)",cg23375068
...,...
"(cg20412955,)",cg20412955
"(cg00359087,)",cg00359087
"(cg10036421,)",cg10036421
"(cg26311705,)",cg26311705


In [68]:
#Generate a new data set with the randomly selected CpGs removed
hannum_test_4 = hannum_test.drop(cpgs_removed_4["CpG"].to_list(), axis=1)

hannum_test_4

CpG,cg00000029,cg00000108,cg00000109,cg00000165,cg00000236,cg00000289,cg00000292,cg00000321,cg00000363,cg00000622,...,ch.9.941347R,ch.9.945770F,ch.9.96055087R,ch.9.98463211R,ch.9.98936572R,ch.9.98937537R,ch.9.98957343R,ch.9.98959675F,ch.9.98989607R,ch.9.991104F
GSM989827,0.464197,0.941091,0.911182,0.132014,0.717861,0.686521,0.805003,0.228244,0.338483,0.016508,...,0.043778,0.022659,0.109918,0.034284,0.133692,0.042808,0.052589,0.035624,0.028066,0.043850
GSM989828,0.454883,0.939033,0.596455,0.206917,0.723935,0.619084,0.814672,0.310879,0.418998,0.005747,...,0.038686,0.005095,0.076996,0.027978,0.125270,0.036811,0.053343,0.075618,0.017428,0.032950
GSM989829,0.485764,0.918802,0.870333,0.162861,0.719196,0.635678,0.824336,0.263215,0.424736,0.012197,...,0.035164,0.021444,0.070694,0.032643,0.139105,0.042844,0.045973,0.126421,0.021752,0.022375
GSM989830,0.480785,0.929908,0.889689,0.197780,0.704061,0.610864,0.811152,0.316761,0.398772,0.019945,...,0.042269,0.028587,0.094749,0.036997,0.140601,0.042258,0.048733,0.084051,0.027504,0.053007
GSM989831,0.501219,0.934548,0.890450,0.148437,0.754913,0.651262,0.808628,0.338289,0.366965,0.000000,...,0.037813,0.018626,0.110543,0.036746,0.129993,0.039613,0.039254,0.165874,0.020889,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM990623,0.527496,0.958173,0.922034,0.223068,0.778959,0.709248,0.825768,0.354296,0.396241,0.001778,...,0.048237,0.016319,0.079741,0.052052,0.159212,0.049571,0.033060,0.093414,0.013654,0.014088
GSM990624,0.588331,0.982450,0.855145,0.162180,0.796869,0.535831,0.806713,0.295598,0.301319,0.006685,...,0.011125,0.023837,0.117762,0.037199,0.133899,0.055904,0.062442,0.101100,0.014193,0.000000
GSM990625,0.362994,0.954392,0.927183,0.196430,0.713020,0.664184,0.804045,0.395724,0.445179,0.003822,...,0.056192,0.012054,0.079249,0.050002,0.149589,0.048526,0.041087,0.049857,0.016840,0.041415
GSM990626,0.499145,0.931691,0.900938,0.167477,0.730215,0.665792,0.831365,0.338117,0.383953,0.000000,...,0.040122,0.009878,0.090751,0.040898,0.168269,0.056429,0.047835,0.028896,0.025346,0.052959


In [69]:
#ensure that all clock CpGs remain in the new set
common = intersection(hannum_test_4.columns.to_list(), sig_cpgs_original["CpG"].to_list())

if not sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    raise ValueError('Some Significant CpGs lost!')

In [70]:
len(common)

847

In [71]:
#Split the dataset into training and test subsets 

methyl_raw_train, methyl_raw_test, age_train, age_test = train_test_split(hannum_test_4, ages, test_size=0.2, random_state=42)


In [72]:
#Scale our data such that the fit is to the training set
scaler = StandardScaler().fit(methyl_raw_train)
methyl_train = scaler.transform(methyl_raw_train)

methyl_test = scaler.transform(methyl_raw_test)

In [73]:
#Read in age training data as a list
Y_train=age_train.values.ravel()

In [74]:
#Create Elastic Net model
elastic_netCV_4_i5000 = ElasticNetCV(l1_ratio = 0.5, n_alphas = 50, cv = 10, n_jobs=3, random_state = 42,
                             max_iter=5000, tol = 0.001, selection='cyclic')

In [75]:
#Train the model. 
elastic_netCV_4_i5000.fit(methyl_train,Y_train)

ElasticNetCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
             l1_ratio=0.5, max_iter=5000, n_alphas=50, n_jobs=3,
             normalize=False, positive=False, precompute='auto',
             random_state=42, selection='cyclic', tol=0.001, verbose=0)

In [76]:
#Save the Elastic Net model
dump(elastic_netCV_4_i5000, parent + '/Trained_Models/elastic_netCV_Hannum_4_i5000.joblib')

['elastic_netCV_Hannum_4_i5000.joblib']

In [77]:
#Save the Dataset
hannum_test_4.to_pickle(parent + '/MethylAndAges/hannum_4% nonsig removed.pkl')

# Now remove 5% CpGs
Here we will remove 5% of nonsig CpGs, create the dataset, and train an EN model

In [78]:
#Find number to be removed
num_removed = int(len_nonsig_original*0.05)

In [79]:
#Generate CpGs to be removed
list_removed = random.sample(range(0, len_nonsig_original), num_removed)

cpgs_removed_5 = nonsig_cpgs_original.iloc[list_removed]
cpgs_removed_5

,CpG
"(cg14360405,)",cg14360405
"(cg18001180,)",cg18001180
"(cg16275894,)",cg16275894
"(cg10421029,)",cg10421029
"(cg05105354,)",cg05105354
...,...
"(cg16456906,)",cg16456906
"(cg20688114,)",cg20688114
"(cg13439478,)",cg13439478
"(cg01827012,)",cg01827012


In [80]:
#Generate a new data set with the randomly selected CpGs removed
hannum_test_5 = hannum_test.drop(cpgs_removed_5["CpG"].to_list(), axis=1)

hannum_test_5

CpG,cg00000029,cg00000108,cg00000109,cg00000165,cg00000236,cg00000289,cg00000292,cg00000321,cg00000363,cg00000622,...,ch.9.945770F,ch.9.96055087R,ch.9.97139671F,ch.9.98463211R,ch.9.98936572R,ch.9.98937537R,ch.9.98957343R,ch.9.98959675F,ch.9.98989607R,ch.9.991104F
GSM989827,0.464197,0.941091,0.911182,0.132014,0.717861,0.686521,0.805003,0.228244,0.338483,0.016508,...,0.022659,0.109918,0.061222,0.034284,0.133692,0.042808,0.052589,0.035624,0.028066,0.043850
GSM989828,0.454883,0.939033,0.596455,0.206917,0.723935,0.619084,0.814672,0.310879,0.418998,0.005747,...,0.005095,0.076996,0.052640,0.027978,0.125270,0.036811,0.053343,0.075618,0.017428,0.032950
GSM989829,0.485764,0.918802,0.870333,0.162861,0.719196,0.635678,0.824336,0.263215,0.424736,0.012197,...,0.021444,0.070694,0.058888,0.032643,0.139105,0.042844,0.045973,0.126421,0.021752,0.022375
GSM989830,0.480785,0.929908,0.889689,0.197780,0.704061,0.610864,0.811152,0.316761,0.398772,0.019945,...,0.028587,0.094749,0.056279,0.036997,0.140601,0.042258,0.048733,0.084051,0.027504,0.053007
GSM989831,0.501219,0.934548,0.890450,0.148437,0.754913,0.651262,0.808628,0.338289,0.366965,0.000000,...,0.018626,0.110543,0.057568,0.036746,0.129993,0.039613,0.039254,0.165874,0.020889,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM990623,0.527496,0.958173,0.922034,0.223068,0.778959,0.709248,0.825768,0.354296,0.396241,0.001778,...,0.016319,0.079741,0.072076,0.052052,0.159212,0.049571,0.033060,0.093414,0.013654,0.014088
GSM990624,0.588331,0.982450,0.855145,0.162180,0.796869,0.535831,0.806713,0.295598,0.301319,0.006685,...,0.023837,0.117762,0.058466,0.037199,0.133899,0.055904,0.062442,0.101100,0.014193,0.000000
GSM990625,0.362994,0.954392,0.927183,0.196430,0.713020,0.664184,0.804045,0.395724,0.445179,0.003822,...,0.012054,0.079249,0.053499,0.050002,0.149589,0.048526,0.041087,0.049857,0.016840,0.041415
GSM990626,0.499145,0.931691,0.900938,0.167477,0.730215,0.665792,0.831365,0.338117,0.383953,0.000000,...,0.009878,0.090751,0.060335,0.040898,0.168269,0.056429,0.047835,0.028896,0.025346,0.052959


In [81]:
#ensure that all clock CpGs remain in the new set
common = intersection(hannum_test_5.columns.to_list(), sig_cpgs_original["CpG"].to_list())

if not sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    raise ValueError('Some Significant CpGs lost!')

In [82]:
len(common)

847

In [83]:
#Split the dataset into training and test subsets 

methyl_raw_train, methyl_raw_test, age_train, age_test = train_test_split(hannum_test_5, ages, test_size=0.2, random_state=42)


In [84]:
#Scale our data such that the fit is to the training set
scaler = StandardScaler().fit(methyl_raw_train)
methyl_train = scaler.transform(methyl_raw_train)

methyl_test = scaler.transform(methyl_raw_test)

In [85]:
#Read in age training data as a list
Y_train=age_train.values.ravel()

In [86]:
#Create Elastic Net model
elastic_netCV_5_i5000 = ElasticNetCV(l1_ratio = 0.5, n_alphas = 50, cv = 10, n_jobs=3, random_state = 42,
                             max_iter=5000, tol = 0.001, selection='cyclic')

In [87]:
#Train the model. 
elastic_netCV_5_i5000.fit(methyl_train,Y_train)

ElasticNetCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
             l1_ratio=0.5, max_iter=5000, n_alphas=50, n_jobs=3,
             normalize=False, positive=False, precompute='auto',
             random_state=42, selection='cyclic', tol=0.001, verbose=0)

In [88]:
#Save the Elastic Net model
dump(elastic_netCV_5_i5000, parent + '/Trained_Models/elastic_netCV_Hannum_5_i5000.joblib')

['elastic_netCV_Hannum_5_i5000.joblib']

In [89]:
#Save the Dataset
hannum_test_5.to_pickle(parent + '/MethylAndAges/hannum_5% nonsig removed.pkl')

# Find significant CpGs and compare
Here, as an assurance that our trainings are working, we will find the new Clock CpGs and compare them to the original

In [90]:
#Get the non-zero coefficients to get the significant cpgs.
coeffs_5_i5000 = pd.DataFrame(elastic_netCV_5_i5000.coef_)
coeffs_5_i5000 = coeffs_5_i5000[(coeffs_5_i5000.T != 0).any()]
coeffs_5_i5000 = coeffs_5_i5000.rename(columns={coeffs_5_i5000.columns[0]: 'Magnitude'})
coeffs_5_i5000

,Magnitude
1233,-0.056068
1286,0.055942
2077,0.031411
2566,0.006534
2617,-0.003707
...,...
447129,-0.020947
447854,0.021009
448637,0.038365
448789,-0.000044


In [91]:
#Get significant CpGs and their indices
colnames_5_i5000 = pd.DataFrame(hannum_test_5.columns)
sig_cpgs_5_i5000 = colnames_5_i5000.iloc[coeffs_5_i5000.index]
sig_cpgs_5_i5000

,CpG
1233,cg00055986
1286,cg00058879
2077,cg00095728
2566,cg00120464
2617,cg00123072
...,...
447129,ch.11.319992F
447854,ch.19.1716179R
448637,ch.4.60646130F
448789,ch.5.67026991F


In [92]:
#Compare original and new Clock CpGs
common = intersection(sig_cpgs_5_i5000["CpG"].to_list(), sig_cpgs_original["CpG"].to_list())

if  sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    print('NO Significant CpGs lost!')

In [93]:
len(common)

689

In [94]:
#Find the original Clock CpGs, which are not clock CpGs in the new model
lost5 = [value for value in sig_cpgs_original["CpG"].to_list() if value not in sig_cpgs_5_i5000["CpG"].to_list()]

In [95]:
len(lost5)

158

# Now remove 6% CpGs
Here we will remove 6% of nonsig CpGs, create the dataset, and train an EN model

In [96]:
#Find number to be removed
num_removed = int(len_nonsig_original*0.06)

In [97]:
#Generate CpGs to be removed
list_removed = random.sample(range(0, len_nonsig_original), num_removed)

cpgs_removed_6 = nonsig_cpgs_original.iloc[list_removed]
cpgs_removed_6

,CpG
"(cg12866229,)",cg12866229
"(cg04450606,)",cg04450606
"(cg00882967,)",cg00882967
"(cg23240895,)",cg23240895
"(cg03696327,)",cg03696327
...,...
"(cg14903245,)",cg14903245
"(cg15060270,)",cg15060270
"(cg05740895,)",cg05740895
"(cg18105767,)",cg18105767


In [98]:
#Generate a new data set with the randomly selected CpGs removed
hannum_test_6 = hannum_test.drop(cpgs_removed_6["CpG"].to_list(), axis=1)

hannum_test_6

CpG,cg00000029,cg00000108,cg00000109,cg00000165,cg00000236,cg00000289,cg00000292,cg00000321,cg00000363,cg00000622,...,ch.9.945770F,ch.9.96055087R,ch.9.97139671F,ch.9.98463211R,ch.9.98936572R,ch.9.98937537R,ch.9.98957343R,ch.9.98959675F,ch.9.98989607R,ch.9.991104F
GSM989827,0.464197,0.941091,0.911182,0.132014,0.717861,0.686521,0.805003,0.228244,0.338483,0.016508,...,0.022659,0.109918,0.061222,0.034284,0.133692,0.042808,0.052589,0.035624,0.028066,0.043850
GSM989828,0.454883,0.939033,0.596455,0.206917,0.723935,0.619084,0.814672,0.310879,0.418998,0.005747,...,0.005095,0.076996,0.052640,0.027978,0.125270,0.036811,0.053343,0.075618,0.017428,0.032950
GSM989829,0.485764,0.918802,0.870333,0.162861,0.719196,0.635678,0.824336,0.263215,0.424736,0.012197,...,0.021444,0.070694,0.058888,0.032643,0.139105,0.042844,0.045973,0.126421,0.021752,0.022375
GSM989830,0.480785,0.929908,0.889689,0.197780,0.704061,0.610864,0.811152,0.316761,0.398772,0.019945,...,0.028587,0.094749,0.056279,0.036997,0.140601,0.042258,0.048733,0.084051,0.027504,0.053007
GSM989831,0.501219,0.934548,0.890450,0.148437,0.754913,0.651262,0.808628,0.338289,0.366965,0.000000,...,0.018626,0.110543,0.057568,0.036746,0.129993,0.039613,0.039254,0.165874,0.020889,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM990623,0.527496,0.958173,0.922034,0.223068,0.778959,0.709248,0.825768,0.354296,0.396241,0.001778,...,0.016319,0.079741,0.072076,0.052052,0.159212,0.049571,0.033060,0.093414,0.013654,0.014088
GSM990624,0.588331,0.982450,0.855145,0.162180,0.796869,0.535831,0.806713,0.295598,0.301319,0.006685,...,0.023837,0.117762,0.058466,0.037199,0.133899,0.055904,0.062442,0.101100,0.014193,0.000000
GSM990625,0.362994,0.954392,0.927183,0.196430,0.713020,0.664184,0.804045,0.395724,0.445179,0.003822,...,0.012054,0.079249,0.053499,0.050002,0.149589,0.048526,0.041087,0.049857,0.016840,0.041415
GSM990626,0.499145,0.931691,0.900938,0.167477,0.730215,0.665792,0.831365,0.338117,0.383953,0.000000,...,0.009878,0.090751,0.060335,0.040898,0.168269,0.056429,0.047835,0.028896,0.025346,0.052959


In [99]:
#ensure that all clock CpGs remain in the new set
common = intersection(hannum_test_6.columns.to_list(), sig_cpgs_original["CpG"].to_list())

if not sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    raise ValueError('Some Significant CpGs lost!')

In [100]:
len(common)

847

In [101]:
#Split the dataset into training and test subsets 

methyl_raw_train, methyl_raw_test, age_train, age_test = train_test_split(hannum_test_6, ages, test_size=0.2, random_state=42)


In [102]:
#Scale our data such that the fit is to the training set
scaler = StandardScaler().fit(methyl_raw_train)
methyl_train = scaler.transform(methyl_raw_train)

methyl_test = scaler.transform(methyl_raw_test)

In [103]:
#Read in age training data as a list
Y_train=age_train.values.ravel()

In [104]:
#Create Elastic Net model
elastic_netCV_6_i5000 = ElasticNetCV(l1_ratio = 0.5, n_alphas = 50, cv = 10, n_jobs=3, random_state = 42,
                             max_iter=5000, tol = 0.001, selection='cyclic')

In [105]:
#Train the model. 
elastic_netCV_6_i5000.fit(methyl_train,Y_train)

ElasticNetCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
             l1_ratio=0.5, max_iter=5000, n_alphas=50, n_jobs=3,
             normalize=False, positive=False, precompute='auto',
             random_state=42, selection='cyclic', tol=0.001, verbose=0)

In [106]:
#Save the Elastic Net model
dump(elastic_netCV_6_i5000, parent + '/Trained_Models/elastic_netCV_Hannum_6_i5000.joblib')

['elastic_netCV_Hannum_6_i5000.joblib']

In [107]:
#Save the Dataset
hannum_test_6.to_pickle(parent + '/MethylAndAges/hannum_6% nonsig removed.pkl')

# Now remove 7% CpGs
Here we will remove 7% of nonsig CpGs, create the dataset, and train an EN model

In [108]:
#Find number to be removed
num_removed = int(len_nonsig_original*0.07)

In [109]:
#Generate CpGs to be removed
list_removed = random.sample(range(0, len_nonsig_original), num_removed)

cpgs_removed_7 = nonsig_cpgs_original.iloc[list_removed]
cpgs_removed_7

,CpG
"(cg20752715,)",cg20752715
"(cg10011792,)",cg10011792
"(cg21068293,)",cg21068293
"(cg14528525,)",cg14528525
"(cg26838532,)",cg26838532
...,...
"(cg09682426,)",cg09682426
"(cg13654797,)",cg13654797
"(cg12420383,)",cg12420383
"(cg01783650,)",cg01783650


In [110]:
#Generate a new data set with the randomly selected CpGs removed
hannum_test_7 = hannum_test.drop(cpgs_removed_7["CpG"].to_list(), axis=1)

hannum_test_7

CpG,cg00000029,cg00000109,cg00000165,cg00000236,cg00000289,cg00000292,cg00000321,cg00000363,cg00000622,cg00000658,...,ch.9.941347R,ch.9.945770F,ch.9.96055087R,ch.9.97139671F,ch.9.98463211R,ch.9.98936572R,ch.9.98957343R,ch.9.98959675F,ch.9.98989607R,ch.9.991104F
GSM989827,0.464197,0.911182,0.132014,0.717861,0.686521,0.805003,0.228244,0.338483,0.016508,0.810140,...,0.043778,0.022659,0.109918,0.061222,0.034284,0.133692,0.052589,0.035624,0.028066,0.043850
GSM989828,0.454883,0.596455,0.206917,0.723935,0.619084,0.814672,0.310879,0.418998,0.005747,0.778277,...,0.038686,0.005095,0.076996,0.052640,0.027978,0.125270,0.053343,0.075618,0.017428,0.032950
GSM989829,0.485764,0.870333,0.162861,0.719196,0.635678,0.824336,0.263215,0.424736,0.012197,0.768844,...,0.035164,0.021444,0.070694,0.058888,0.032643,0.139105,0.045973,0.126421,0.021752,0.022375
GSM989830,0.480785,0.889689,0.197780,0.704061,0.610864,0.811152,0.316761,0.398772,0.019945,0.825187,...,0.042269,0.028587,0.094749,0.056279,0.036997,0.140601,0.048733,0.084051,0.027504,0.053007
GSM989831,0.501219,0.890450,0.148437,0.754913,0.651262,0.808628,0.338289,0.366965,0.000000,0.816176,...,0.037813,0.018626,0.110543,0.057568,0.036746,0.129993,0.039254,0.165874,0.020889,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM990623,0.527496,0.922034,0.223068,0.778959,0.709248,0.825768,0.354296,0.396241,0.001778,0.825719,...,0.048237,0.016319,0.079741,0.072076,0.052052,0.159212,0.033060,0.093414,0.013654,0.014088
GSM990624,0.588331,0.855145,0.162180,0.796869,0.535831,0.806713,0.295598,0.301319,0.006685,0.821282,...,0.011125,0.023837,0.117762,0.058466,0.037199,0.133899,0.062442,0.101100,0.014193,0.000000
GSM990625,0.362994,0.927183,0.196430,0.713020,0.664184,0.804045,0.395724,0.445179,0.003822,0.810389,...,0.056192,0.012054,0.079249,0.053499,0.050002,0.149589,0.041087,0.049857,0.016840,0.041415
GSM990626,0.499145,0.900938,0.167477,0.730215,0.665792,0.831365,0.338117,0.383953,0.000000,0.864885,...,0.040122,0.009878,0.090751,0.060335,0.040898,0.168269,0.047835,0.028896,0.025346,0.052959


In [111]:
#ensure that all clock CpGs remain in the new set
common = intersection(hannum_test_7.columns.to_list(), sig_cpgs_original["CpG"].to_list())

if not sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    raise ValueError('Some Significant CpGs lost!')

In [112]:
len(common)

847

In [113]:
#Split the dataset into training and test subsets 

methyl_raw_train, methyl_raw_test, age_train, age_test = train_test_split(hannum_test_7, ages, test_size=0.2, random_state=42)


In [114]:
#Scale our data such that the fit is to the training set
scaler = StandardScaler().fit(methyl_raw_train)
methyl_train = scaler.transform(methyl_raw_train)

methyl_test = scaler.transform(methyl_raw_test)

In [115]:
#Read in age training data as a list
Y_train=age_train.values.ravel()

In [116]:
#Create Elastic Net model
elastic_netCV_7_i5000 = ElasticNetCV(l1_ratio = 0.5, n_alphas = 50, cv = 10, n_jobs=3, random_state = 42,
                             max_iter=5000, tol = 0.001, selection='cyclic')

In [117]:
#Train the model. 
elastic_netCV_7_i5000.fit(methyl_train,Y_train)

ElasticNetCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
             l1_ratio=0.5, max_iter=5000, n_alphas=50, n_jobs=3,
             normalize=False, positive=False, precompute='auto',
             random_state=42, selection='cyclic', tol=0.001, verbose=0)

In [118]:
#Save the Elastic Net model
dump(elastic_netCV_7_i5000, parent + '/Trained_Models/elastic_netCV_Hannum_7_i5000.joblib')

['elastic_netCV_Hannum_7_i5000.joblib']

In [119]:
#Save the Dataset
hannum_test_7.to_pickle(parent + '/MethylAndAges/hannum_7% nonsig removed.pkl')

# Now remove 8% CpGs
Here we will remove 8% of nonsig CpGs, create the dataset, and train an EN model

In [120]:
#Find number to be removed
num_removed = int(len_nonsig_original*0.08)

In [121]:
#Generate CpGs to be removed
list_removed = random.sample(range(0, len_nonsig_original), num_removed)

cpgs_removed_8 = nonsig_cpgs_original.iloc[list_removed]
cpgs_removed_8

,CpG
"(cg05163663,)",cg05163663
"(ch.8.120063967R,)",ch.8.120063967R
"(cg15456082,)",cg15456082
"(cg06364593,)",cg06364593
"(cg26001719,)",cg26001719
...,...
"(cg14446165,)",cg14446165
"(cg23094620,)",cg23094620
"(cg21385522,)",cg21385522
"(cg19219437,)",cg19219437


In [122]:
#Generate a new data set with the randomly selected CpGs removed
hannum_test_8 = hannum_test.drop(cpgs_removed_8["CpG"].to_list(), axis=1)

hannum_test_8

CpG,cg00000029,cg00000108,cg00000109,cg00000165,cg00000236,cg00000289,cg00000292,cg00000321,cg00000363,cg00000622,...,ch.9.945770F,ch.9.96055087R,ch.9.97139671F,ch.9.98463211R,ch.9.98936572R,ch.9.98937537R,ch.9.98957343R,ch.9.98959675F,ch.9.98989607R,ch.9.991104F
GSM989827,0.464197,0.941091,0.911182,0.132014,0.717861,0.686521,0.805003,0.228244,0.338483,0.016508,...,0.022659,0.109918,0.061222,0.034284,0.133692,0.042808,0.052589,0.035624,0.028066,0.043850
GSM989828,0.454883,0.939033,0.596455,0.206917,0.723935,0.619084,0.814672,0.310879,0.418998,0.005747,...,0.005095,0.076996,0.052640,0.027978,0.125270,0.036811,0.053343,0.075618,0.017428,0.032950
GSM989829,0.485764,0.918802,0.870333,0.162861,0.719196,0.635678,0.824336,0.263215,0.424736,0.012197,...,0.021444,0.070694,0.058888,0.032643,0.139105,0.042844,0.045973,0.126421,0.021752,0.022375
GSM989830,0.480785,0.929908,0.889689,0.197780,0.704061,0.610864,0.811152,0.316761,0.398772,0.019945,...,0.028587,0.094749,0.056279,0.036997,0.140601,0.042258,0.048733,0.084051,0.027504,0.053007
GSM989831,0.501219,0.934548,0.890450,0.148437,0.754913,0.651262,0.808628,0.338289,0.366965,0.000000,...,0.018626,0.110543,0.057568,0.036746,0.129993,0.039613,0.039254,0.165874,0.020889,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM990623,0.527496,0.958173,0.922034,0.223068,0.778959,0.709248,0.825768,0.354296,0.396241,0.001778,...,0.016319,0.079741,0.072076,0.052052,0.159212,0.049571,0.033060,0.093414,0.013654,0.014088
GSM990624,0.588331,0.982450,0.855145,0.162180,0.796869,0.535831,0.806713,0.295598,0.301319,0.006685,...,0.023837,0.117762,0.058466,0.037199,0.133899,0.055904,0.062442,0.101100,0.014193,0.000000
GSM990625,0.362994,0.954392,0.927183,0.196430,0.713020,0.664184,0.804045,0.395724,0.445179,0.003822,...,0.012054,0.079249,0.053499,0.050002,0.149589,0.048526,0.041087,0.049857,0.016840,0.041415
GSM990626,0.499145,0.931691,0.900938,0.167477,0.730215,0.665792,0.831365,0.338117,0.383953,0.000000,...,0.009878,0.090751,0.060335,0.040898,0.168269,0.056429,0.047835,0.028896,0.025346,0.052959


In [123]:
#ensure that all clock CpGs remain in the new set
common = intersection(hannum_test_8.columns.to_list(), sig_cpgs_original["CpG"].to_list())

if not sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    raise ValueError('Some Significant CpGs lost!')

In [124]:
len(common)

847

In [125]:
#Split the dataset into training and test subsets 

methyl_raw_train, methyl_raw_test, age_train, age_test = train_test_split(hannum_test_8, ages, test_size=0.2, random_state=42)


In [126]:
#Scale our data such that the fit is to the training set
scaler = StandardScaler().fit(methyl_raw_train)
methyl_train = scaler.transform(methyl_raw_train)

methyl_test = scaler.transform(methyl_raw_test)

In [127]:
#Read in age training data as a list
Y_train=age_train.values.ravel()

In [128]:
#Create Elastic Net model
elastic_netCV_8_i5000 = ElasticNetCV(l1_ratio = 0.5, n_alphas = 50, cv = 10, n_jobs=3, random_state = 42,
                             max_iter=5000, tol = 0.001, selection='cyclic')

In [129]:
#Train the model. 
elastic_netCV_8_i5000.fit(methyl_train,Y_train)

ElasticNetCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
             l1_ratio=0.5, max_iter=5000, n_alphas=50, n_jobs=3,
             normalize=False, positive=False, precompute='auto',
             random_state=42, selection='cyclic', tol=0.001, verbose=0)

In [130]:
#Save the Elastic Net model
dump(elastic_netCV_8_i5000, parent + '/Trained_Models/elastic_netCV_Hannum_8_i5000.joblib')

['elastic_netCV_Hannum_8_i5000.joblib']

In [131]:
#Save the Dataset
hannum_test_8.to_pickle(parent + '/MethylAndAges/hannum_8% nonsig removed.pkl')

# Now remove 9% CpGs
Here we will remove 9% of nonsig CpGs, create the dataset, and train an EN model

In [132]:
#Find number to be removed
num_removed = int(len_nonsig_original*0.09)

In [133]:
#Generate CpGs to be removed
list_removed = random.sample(range(0, len_nonsig_original), num_removed)

cpgs_removed_9 = nonsig_cpgs_original.iloc[list_removed]
cpgs_removed_9

,CpG
"(cg13033578,)",cg13033578
"(cg25520146,)",cg25520146
"(cg14240634,)",cg14240634
"(cg22615510,)",cg22615510
"(cg05376648,)",cg05376648
...,...
"(cg18007858,)",cg18007858
"(cg24818238,)",cg24818238
"(cg24524546,)",cg24524546
"(cg00931520,)",cg00931520


In [134]:
#Generate a new data set with the randomly selected CpGs removed
hannum_test_9 = hannum_test.drop(cpgs_removed_9["CpG"].to_list(), axis=1)

hannum_test_9

CpG,cg00000029,cg00000108,cg00000109,cg00000236,cg00000289,cg00000292,cg00000321,cg00000363,cg00000622,cg00000658,...,ch.9.941347R,ch.9.945770F,ch.9.96055087R,ch.9.97139671F,ch.9.98463211R,ch.9.98936572R,ch.9.98937537R,ch.9.98959675F,ch.9.98989607R,ch.9.991104F
GSM989827,0.464197,0.941091,0.911182,0.717861,0.686521,0.805003,0.228244,0.338483,0.016508,0.810140,...,0.043778,0.022659,0.109918,0.061222,0.034284,0.133692,0.042808,0.035624,0.028066,0.043850
GSM989828,0.454883,0.939033,0.596455,0.723935,0.619084,0.814672,0.310879,0.418998,0.005747,0.778277,...,0.038686,0.005095,0.076996,0.052640,0.027978,0.125270,0.036811,0.075618,0.017428,0.032950
GSM989829,0.485764,0.918802,0.870333,0.719196,0.635678,0.824336,0.263215,0.424736,0.012197,0.768844,...,0.035164,0.021444,0.070694,0.058888,0.032643,0.139105,0.042844,0.126421,0.021752,0.022375
GSM989830,0.480785,0.929908,0.889689,0.704061,0.610864,0.811152,0.316761,0.398772,0.019945,0.825187,...,0.042269,0.028587,0.094749,0.056279,0.036997,0.140601,0.042258,0.084051,0.027504,0.053007
GSM989831,0.501219,0.934548,0.890450,0.754913,0.651262,0.808628,0.338289,0.366965,0.000000,0.816176,...,0.037813,0.018626,0.110543,0.057568,0.036746,0.129993,0.039613,0.165874,0.020889,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM990623,0.527496,0.958173,0.922034,0.778959,0.709248,0.825768,0.354296,0.396241,0.001778,0.825719,...,0.048237,0.016319,0.079741,0.072076,0.052052,0.159212,0.049571,0.093414,0.013654,0.014088
GSM990624,0.588331,0.982450,0.855145,0.796869,0.535831,0.806713,0.295598,0.301319,0.006685,0.821282,...,0.011125,0.023837,0.117762,0.058466,0.037199,0.133899,0.055904,0.101100,0.014193,0.000000
GSM990625,0.362994,0.954392,0.927183,0.713020,0.664184,0.804045,0.395724,0.445179,0.003822,0.810389,...,0.056192,0.012054,0.079249,0.053499,0.050002,0.149589,0.048526,0.049857,0.016840,0.041415
GSM990626,0.499145,0.931691,0.900938,0.730215,0.665792,0.831365,0.338117,0.383953,0.000000,0.864885,...,0.040122,0.009878,0.090751,0.060335,0.040898,0.168269,0.056429,0.028896,0.025346,0.052959


In [135]:
#ensure that all clock CpGs remain in the new set
common = intersection(hannum_test_9.columns.to_list(), sig_cpgs_original["CpG"].to_list())

if not sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    raise ValueError('Some Significant CpGs lost!')

In [136]:
len(common)

847

In [137]:
#Split the dataset into training and test subsets 

methyl_raw_train, methyl_raw_test, age_train, age_test = train_test_split(hannum_test_9, ages, test_size=0.2, random_state=42)


In [138]:
#Scale our data such that the fit is to the training set
scaler = StandardScaler().fit(methyl_raw_train)
methyl_train = scaler.transform(methyl_raw_train)

methyl_test = scaler.transform(methyl_raw_test)

In [139]:
#Read in age training data as a list
Y_train=age_train.values.ravel()

In [140]:
#Create Elastic Net model
elastic_netCV_9_i5000 = ElasticNetCV(l1_ratio = 0.5, n_alphas = 50, cv = 10, n_jobs=3, random_state = 42,
                             max_iter=5000, tol = 0.001, selection='cyclic')

In [141]:
#Train the model. 
elastic_netCV_9_i5000.fit(methyl_train,Y_train)

ElasticNetCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
             l1_ratio=0.5, max_iter=5000, n_alphas=50, n_jobs=3,
             normalize=False, positive=False, precompute='auto',
             random_state=42, selection='cyclic', tol=0.001, verbose=0)

In [142]:
#Save the Elastic Net model
dump(elastic_netCV_9_i5000, parent + '/Trained_Models/elastic_netCV_Hannum_9_i5000.joblib')

['elastic_netCV_Hannum_9_i5000.joblib']

In [143]:
#Save the Dataset
hannum_test_9.to_pickle(parent + '/MethylAndAges/hannum_9% nonsig removed.pkl')

# Now remove 10% CpGs
Here we will remove 10% of nonsig CpGs, create the dataset, and train an EN model

In [144]:
#Find number to be removed
num_removed = int(len_nonsig_original*0.1)

In [145]:
#Generate CpGs to be removed
list_removed = random.sample(range(0, len_nonsig_original), num_removed)

cpgs_removed_10 = nonsig_cpgs_original.iloc[list_removed]
cpgs_removed_10

,CpG
"(cg24083746,)",cg24083746
"(cg13842222,)",cg13842222
"(cg12723994,)",cg12723994
"(cg08121890,)",cg08121890
"(cg18226335,)",cg18226335
...,...
"(cg02376916,)",cg02376916
"(cg00173799,)",cg00173799
"(cg16663025,)",cg16663025
"(cg11140778,)",cg11140778


In [146]:
#Generate a new data set with the randomly selected CpGs removed
hannum_test_10 = hannum_test.drop(cpgs_removed_10["CpG"].to_list(), axis=1)

hannum_test_10

CpG,cg00000029,cg00000108,cg00000109,cg00000165,cg00000236,cg00000289,cg00000292,cg00000321,cg00000622,cg00000658,...,ch.9.941347R,ch.9.945770F,ch.9.96055087R,ch.9.97139671F,ch.9.98936572R,ch.9.98937537R,ch.9.98957343R,ch.9.98959675F,ch.9.98989607R,ch.9.991104F
GSM989827,0.464197,0.941091,0.911182,0.132014,0.717861,0.686521,0.805003,0.228244,0.016508,0.810140,...,0.043778,0.022659,0.109918,0.061222,0.133692,0.042808,0.052589,0.035624,0.028066,0.043850
GSM989828,0.454883,0.939033,0.596455,0.206917,0.723935,0.619084,0.814672,0.310879,0.005747,0.778277,...,0.038686,0.005095,0.076996,0.052640,0.125270,0.036811,0.053343,0.075618,0.017428,0.032950
GSM989829,0.485764,0.918802,0.870333,0.162861,0.719196,0.635678,0.824336,0.263215,0.012197,0.768844,...,0.035164,0.021444,0.070694,0.058888,0.139105,0.042844,0.045973,0.126421,0.021752,0.022375
GSM989830,0.480785,0.929908,0.889689,0.197780,0.704061,0.610864,0.811152,0.316761,0.019945,0.825187,...,0.042269,0.028587,0.094749,0.056279,0.140601,0.042258,0.048733,0.084051,0.027504,0.053007
GSM989831,0.501219,0.934548,0.890450,0.148437,0.754913,0.651262,0.808628,0.338289,0.000000,0.816176,...,0.037813,0.018626,0.110543,0.057568,0.129993,0.039613,0.039254,0.165874,0.020889,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM990623,0.527496,0.958173,0.922034,0.223068,0.778959,0.709248,0.825768,0.354296,0.001778,0.825719,...,0.048237,0.016319,0.079741,0.072076,0.159212,0.049571,0.033060,0.093414,0.013654,0.014088
GSM990624,0.588331,0.982450,0.855145,0.162180,0.796869,0.535831,0.806713,0.295598,0.006685,0.821282,...,0.011125,0.023837,0.117762,0.058466,0.133899,0.055904,0.062442,0.101100,0.014193,0.000000
GSM990625,0.362994,0.954392,0.927183,0.196430,0.713020,0.664184,0.804045,0.395724,0.003822,0.810389,...,0.056192,0.012054,0.079249,0.053499,0.149589,0.048526,0.041087,0.049857,0.016840,0.041415
GSM990626,0.499145,0.931691,0.900938,0.167477,0.730215,0.665792,0.831365,0.338117,0.000000,0.864885,...,0.040122,0.009878,0.090751,0.060335,0.168269,0.056429,0.047835,0.028896,0.025346,0.052959


In [147]:
#ensure that all clock CpGs remain in the new set
common = intersection(hannum_test_10.columns.to_list(), sig_cpgs_original["CpG"].to_list())

if not sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    raise ValueError('Some Significant CpGs lost!')

In [148]:
#Split the dataset into training and test subsets 

methyl_raw_train, methyl_raw_test, age_train, age_test = train_test_split(hannum_test_10, ages, test_size=0.2, random_state=42)


In [149]:
#Scale our data such that the fit is to the training set
scaler = StandardScaler().fit(methyl_raw_train)
methyl_train = scaler.transform(methyl_raw_train)

methyl_test = scaler.transform(methyl_raw_test)

In [150]:
#Read in age training data as a list
Y_train=age_train.values.ravel()

In [151]:
#Create Elastic Net model
elastic_netCV_10_i5000 = ElasticNetCV(l1_ratio = 0.5, n_alphas = 50, cv = 10, n_jobs=3, random_state = 42,
                             max_iter=5000, tol = 0.001, selection='cyclic')

In [152]:
#Train the model.
elastic_netCV_10_i5000.fit(methyl_train,Y_train)

ElasticNetCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
             l1_ratio=0.5, max_iter=5000, n_alphas=50, n_jobs=3,
             normalize=False, positive=False, precompute='auto',
             random_state=42, selection='cyclic', tol=0.001, verbose=0)

In [153]:
#Save the Elastic Net model
dump(elastic_netCV_10_i5000, parent + '/Trained_Models/elastic_netCV_Hannum_10_i5000.joblib')

['elastic_netCV_Hannum_10_i5000.joblib']

In [154]:
#Save the Dataset
hannum_test_10.to_pickle(parent + '/MethylAndAges/hannum_10% nonsig removed.pkl')

# Find significant CpGs and compare
Here, as an assurance that our trainings are working, we will find the new Clock CpGs and compare them to the original

In [155]:
#Get the non-zero coefficients to get the significant cpgs. Runtime ~3 min
coeffs_10_i5000 = pd.DataFrame(elastic_netCV_10_i5000.coef_)
coeffs_10_i5000 = coeffs_10_i5000[(coeffs_10_i5000.T != 0).any()]
coeffs_10_i5000

,0
1162,-0.056720
1215,0.065293
1965,0.038605
2486,-0.000346
2887,0.034979
...,...
423638,-0.156757
423639,-0.017862
424328,0.008536
425084,0.034846


In [156]:
#Get significant CpGs and their indices
colnames_10_i5000 = pd.DataFrame(hannum_test_10.columns)
sig_cpgs_10_i5000 = colnames_10_i5000.iloc[coeffs_10_i5000.index]
sig_cpgs_10_i5000

,CpG
1162,cg00055986
1215,cg00058879
1965,cg00095728
2486,cg00123072
2887,cg00147095
...,...
423638,ch.11.315572F
423639,ch.11.319992F
424328,ch.19.1716179R
425084,ch.4.60646130F


In [157]:
#Compare original and new Clock CpGs
common = intersection(sig_cpgs_10_i5000["CpG"].to_list(), sig_cpgs_original["CpG"].to_list())

if  sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    print('NO Significant CpGs lost!')

In [158]:
len(common)

684

# Now remove 20% CpGs
Here we will remove 20% of nonsig CpGs, create the dataset, and train an EN model

In [159]:
#Find number to be removed
num_removed = int(len_nonsig_original*0.20)

In [160]:
#Generate CpGs to be removed
list_removed = random.sample(range(0, len_nonsig_original), num_removed)

cpgs_removed_20 = nonsig_cpgs_original.iloc[list_removed]
cpgs_removed_20

,CpG
"(cg08172947,)",cg08172947
"(cg06918925,)",cg06918925
"(cg14408823,)",cg14408823
"(cg27619136,)",cg27619136
"(cg07921371,)",cg07921371
...,...
"(cg20959920,)",cg20959920
"(cg02119955,)",cg02119955
"(cg03030717,)",cg03030717
"(cg25062778,)",cg25062778


In [161]:
#Generate a new data set with the randomly selected CpGs removed
hannum_test_20 = hannum_test.drop(cpgs_removed_20["CpG"].to_list(), axis=1)

hannum_test_20

CpG,cg00000029,cg00000108,cg00000109,cg00000165,cg00000236,cg00000292,cg00000321,cg00000363,cg00000622,cg00000658,...,ch.9.941347R,ch.9.945770F,ch.9.97139671F,ch.9.98463211R,ch.9.98936572R,ch.9.98937537R,ch.9.98957343R,ch.9.98959675F,ch.9.98989607R,ch.9.991104F
GSM989827,0.464197,0.941091,0.911182,0.132014,0.717861,0.805003,0.228244,0.338483,0.016508,0.810140,...,0.043778,0.022659,0.061222,0.034284,0.133692,0.042808,0.052589,0.035624,0.028066,0.043850
GSM989828,0.454883,0.939033,0.596455,0.206917,0.723935,0.814672,0.310879,0.418998,0.005747,0.778277,...,0.038686,0.005095,0.052640,0.027978,0.125270,0.036811,0.053343,0.075618,0.017428,0.032950
GSM989829,0.485764,0.918802,0.870333,0.162861,0.719196,0.824336,0.263215,0.424736,0.012197,0.768844,...,0.035164,0.021444,0.058888,0.032643,0.139105,0.042844,0.045973,0.126421,0.021752,0.022375
GSM989830,0.480785,0.929908,0.889689,0.197780,0.704061,0.811152,0.316761,0.398772,0.019945,0.825187,...,0.042269,0.028587,0.056279,0.036997,0.140601,0.042258,0.048733,0.084051,0.027504,0.053007
GSM989831,0.501219,0.934548,0.890450,0.148437,0.754913,0.808628,0.338289,0.366965,0.000000,0.816176,...,0.037813,0.018626,0.057568,0.036746,0.129993,0.039613,0.039254,0.165874,0.020889,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM990623,0.527496,0.958173,0.922034,0.223068,0.778959,0.825768,0.354296,0.396241,0.001778,0.825719,...,0.048237,0.016319,0.072076,0.052052,0.159212,0.049571,0.033060,0.093414,0.013654,0.014088
GSM990624,0.588331,0.982450,0.855145,0.162180,0.796869,0.806713,0.295598,0.301319,0.006685,0.821282,...,0.011125,0.023837,0.058466,0.037199,0.133899,0.055904,0.062442,0.101100,0.014193,0.000000
GSM990625,0.362994,0.954392,0.927183,0.196430,0.713020,0.804045,0.395724,0.445179,0.003822,0.810389,...,0.056192,0.012054,0.053499,0.050002,0.149589,0.048526,0.041087,0.049857,0.016840,0.041415
GSM990626,0.499145,0.931691,0.900938,0.167477,0.730215,0.831365,0.338117,0.383953,0.000000,0.864885,...,0.040122,0.009878,0.060335,0.040898,0.168269,0.056429,0.047835,0.028896,0.025346,0.052959


In [162]:
#ensure that all clock CpGs remain in the new set
common = intersection(hannum_test_20.columns.to_list(), sig_cpgs_original["CpG"].to_list())

if not sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    raise ValueError('Some Significant CpGs lost!')

In [163]:
len(common)

847

In [164]:
#Split the dataset into training and test subsets 

methyl_raw_train, methyl_raw_test, age_train, age_test = train_test_split(hannum_test_20, ages, test_size=0.2, random_state=42)


In [165]:
#Scale our data such that the fit is to the training set
scaler = StandardScaler().fit(methyl_raw_train)
methyl_train = scaler.transform(methyl_raw_train)

methyl_test = scaler.transform(methyl_raw_test)

In [166]:
#Read in age training data as a list
Y_train=age_train.values.ravel()

In [167]:
#Create Elastic Net model
elastic_netCV_20_i5000 = ElasticNetCV(l1_ratio = 0.5, n_alphas = 50, cv = 10, n_jobs=3, random_state = 42,
                             max_iter=5000, tol = 0.001, selection='cyclic')

In [168]:
#Train the model.  
elastic_netCV_20_i5000.fit(methyl_train,Y_train)

ElasticNetCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
             l1_ratio=0.5, max_iter=5000, n_alphas=50, n_jobs=3,
             normalize=False, positive=False, precompute='auto',
             random_state=42, selection='cyclic', tol=0.001, verbose=0)

In [169]:
#Save the Elastic Net model
dump(elastic_netCV_20_i5000, parent + '/Trained_Models/elastic_netCV_Hannum_20_i5000.joblib')

['D:\\elastic_netCV_Hannum_20_i5000.joblib']

In [170]:
#Save the Dataset
hannum_test_20.to_pickle(parent + '/MethylAndAges/hannum_20% nonsig removed.pkl')

# Find significant CpGs and compare
Here, as an assurance that our trainings are working, we will find the new Clock CpGs and compare them to the original

In [171]:
#Get the non-zero coefficients to get the significant cpgs. Runtime ~3 min
coeffs_20_i5000 = pd.DataFrame(elastic_netCV_20_i5000.coef_)
coeffs_20_i5000 = coeffs_20_i5000[(coeffs_20_i5000.T != 0).any()]
coeffs_20_i5000

,0
68,-0.015230
809,0.002759
877,-0.015692
1018,-0.042531
1068,0.038483
...,...
376087,-0.044202
376608,-0.141177
377241,0.034094
377921,0.047199


In [172]:
#Get significant CpGs and their indices
colnames_20_i5000 = pd.DataFrame(hannum_test_20.columns)
sig_cpgs_20_i5000 = colnames_20_i5000.iloc[coeffs_20_i5000.index]
sig_cpgs_20_i5000

,CpG
68,cg00003722
809,cg00042478
877,cg00047050
1018,cg00055986
1068,cg00058879
...,...
376087,cg27662246
376608,ch.11.315572F
377241,ch.19.1716179R
377921,ch.4.60646130F


In [173]:
#Compare original and new Clock CpGs
common = intersection(sig_cpgs_20_i5000["CpG"].to_list(), sig_cpgs_original["CpG"].to_list())

if  sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    print('NO Significant CpGs lost!')

In [174]:
len(common)

702

In [175]:
#Find the original Clock CpGs, which are not clock CpGs in the new model
loss = [value for value in sig_cpgs_original["CpG"].to_list() if value not in sig_cpgs_20_i5000["CpG"].to_list()]
len(loss)

145

# Now remove 30% CpGs
Here we will remove 30% of nonsig CpGs, create the dataset, and train an EN model

In [176]:
#Find number to be removed
num_removed = int(len_nonsig_original*0.30)

In [177]:
#Generate CpGs to be removed
list_removed = random.sample(range(0, len_nonsig_original), num_removed)

cpgs_removed_30 = nonsig_cpgs_original.iloc[list_removed]
cpgs_removed_30

,CpG
"(cg03692708,)",cg03692708
"(cg15864491,)",cg15864491
"(cg17808631,)",cg17808631
"(cg24278817,)",cg24278817
"(cg18901140,)",cg18901140
...,...
"(cg24663636,)",cg24663636
"(cg03459811,)",cg03459811
"(cg08318424,)",cg08318424
"(cg06954520,)",cg06954520


In [178]:
#Generate a new data set with the randomly selected CpGs removed
hannum_test_30 = hannum_test.drop(cpgs_removed_30["CpG"].to_list(), axis=1)

hannum_test_30

CpG,cg00000029,cg00000109,cg00000236,cg00000289,cg00000321,cg00000363,cg00000622,cg00000658,cg00000714,cg00000721,...,ch.9.919537F,ch.9.93373462R,ch.9.93402636R,ch.9.96055087R,ch.9.97139671F,ch.9.98463211R,ch.9.98937537R,ch.9.98957343R,ch.9.98959675F,ch.9.98989607R
GSM989827,0.464197,0.911182,0.717861,0.686521,0.228244,0.338483,0.016508,0.810140,0.177981,0.921818,...,0.092026,0.013003,0.025619,0.109918,0.061222,0.034284,0.042808,0.052589,0.035624,0.028066
GSM989828,0.454883,0.596455,0.723935,0.619084,0.310879,0.418998,0.005747,0.778277,0.144454,0.907529,...,0.114144,0.007174,0.029295,0.076996,0.052640,0.027978,0.036811,0.053343,0.075618,0.017428
GSM989829,0.485764,0.870333,0.719196,0.635678,0.263215,0.424736,0.012197,0.768844,0.185125,0.916278,...,0.110516,0.014274,0.031233,0.070694,0.058888,0.032643,0.042844,0.045973,0.126421,0.021752
GSM989830,0.480785,0.889689,0.704061,0.610864,0.316761,0.398772,0.019945,0.825187,0.162875,0.913187,...,0.102213,0.014307,0.035059,0.094749,0.056279,0.036997,0.042258,0.048733,0.084051,0.027504
GSM989831,0.501219,0.890450,0.754913,0.651262,0.338289,0.366965,0.000000,0.816176,0.198095,0.924478,...,0.111282,0.017529,0.026308,0.110543,0.057568,0.036746,0.039613,0.039254,0.165874,0.020889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM990623,0.527496,0.922034,0.778959,0.709248,0.354296,0.396241,0.001778,0.825719,0.203130,0.946202,...,0.120232,0.009137,0.023085,0.079741,0.072076,0.052052,0.049571,0.033060,0.093414,0.013654
GSM990624,0.588331,0.855145,0.796869,0.535831,0.295598,0.301319,0.006685,0.821282,0.228045,0.943319,...,0.121977,0.018655,0.029672,0.117762,0.058466,0.037199,0.055904,0.062442,0.101100,0.014193
GSM990625,0.362994,0.927183,0.713020,0.664184,0.395724,0.445179,0.003822,0.810389,0.205631,0.939465,...,0.147659,0.030143,0.031308,0.079249,0.053499,0.050002,0.048526,0.041087,0.049857,0.016840
GSM990626,0.499145,0.900938,0.730215,0.665792,0.338117,0.383953,0.000000,0.864885,0.234228,0.959109,...,0.145616,0.032191,0.045281,0.090751,0.060335,0.040898,0.056429,0.047835,0.028896,0.025346


In [179]:
#ensure that all clock CpGs remain in the new set
common = intersection(hannum_test_30.columns.to_list(), sig_cpgs_original["CpG"].to_list())

if not sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    raise ValueError('Some Significant CpGs lost!')

In [180]:
len(common)

847

In [181]:
#Split the dataset into training and test subsets 

methyl_raw_train, methyl_raw_test, age_train, age_test = train_test_split(hannum_test_30, ages, test_size=0.2, random_state=42)


In [182]:
#Scale our data such that the fit is to the training set
scaler = StandardScaler().fit(methyl_raw_train)
methyl_train = scaler.transform(methyl_raw_train)

methyl_test = scaler.transform(methyl_raw_test)

In [183]:
#Read in age training data as a list
Y_train=age_train.values.ravel()

In [184]:
#Create Elastic Net model
elastic_netCV_30_i5000 = ElasticNetCV(l1_ratio = 0.5, n_alphas = 50, cv = 10, n_jobs=3, random_state = 42,
                             max_iter=5000, tol = 0.001, selection='cyclic')

In [185]:
#Train the model. 
elastic_netCV_30_i5000.fit(methyl_train,Y_train)

ElasticNetCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
             l1_ratio=0.5, max_iter=5000, n_alphas=50, n_jobs=3,
             normalize=False, positive=False, precompute='auto',
             random_state=42, selection='cyclic', tol=0.001, verbose=0)

In [186]:
#Save the Elastic Net model
dump(elastic_netCV_30_i5000, parent + '/Trained_Models/elastic_netCV_Hannum_30_i5000.joblib')

['D:\\elastic_netCV_Hannum_30_i5000.joblib']

In [188]:
#Save the Dataset
hannum_test_30.to_pickle(parent + '/MethylAndAges/hannum_30% nonsig removed.pkl')

# Find significant CpGs and compare
Here, as an assurance that our trainings are working, we will find the new Clock CpGs and compare them to the original

In [190]:
#Get the non-zero coefficients to get the significant cpgs. Runtime ~3 min
coeffs_30_i5000 = pd.DataFrame(elastic_netCV_30_i5000.coef_)
coeffs_30_i5000 = coeffs_30_i5000[(coeffs_30_i5000.T != 0).any()]
coeffs_30_i5000

,0
726,0.009281
792,-0.000002
926,-0.060995
976,0.053213
1574,0.042574
...,...
329663,-0.017207
330203,0.016147
330786,0.045742
330904,-0.004893


In [191]:
#Get significant CpGs and their indices
colnames_30_i5000 = pd.DataFrame(hannum_test_30.columns)
sig_cpgs_30_i5000 = colnames_30_i5000.iloc[coeffs_30_i5000.index]
sig_cpgs_30_i5000

,CpG
726,cg00042478
792,cg00047050
926,cg00055986
976,cg00058879
1574,cg00095728
...,...
329663,ch.11.319992F
330203,ch.19.1716179R
330786,ch.4.60646130F
330904,ch.5.67026991F


In [192]:
#Compare original and new Clock CpGs
common = intersection(sig_cpgs_30_i5000["CpG"].to_list(), sig_cpgs_original["CpG"].to_list())

if  sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    print('NO Significant CpGs lost!')

In [193]:
len(common)

691

In [194]:
#Find the original Clock CpGs, which are not clock CpGs in the new model
loss = [value for value in sig_cpgs_original["CpG"].to_list() if value not in sig_cpgs_30_i5000["CpG"].to_list()]
len(loss)

156

# Now remove 40% CpGs
Here we will remove 40% of nonsig CpGs, create the dataset, and train an EN model

In [195]:
#Find number to be removed
num_removed = int(len_nonsig_original*0.40)

In [196]:
#Generate CpGs to be removed
list_removed = random.sample(range(0, len_nonsig_original), num_removed)

cpgs_removed_40 = nonsig_cpgs_original.iloc[list_removed]
cpgs_removed_40

,CpG
"(cg19438894,)",cg19438894
"(cg03879460,)",cg03879460
"(cg01922090,)",cg01922090
"(cg17993282,)",cg17993282
"(cg19505136,)",cg19505136
...,...
"(cg12775909,)",cg12775909
"(cg20514973,)",cg20514973
"(cg24096387,)",cg24096387
"(cg27522780,)",cg27522780


In [197]:
#Generate a new data set with the randomly selected CpGs removed
hannum_test_40 = hannum_test.drop(cpgs_removed_40["CpG"].to_list(), axis=1)

hannum_test_40

CpG,cg00000029,cg00000108,cg00000165,cg00000289,cg00000321,cg00000714,cg00000721,cg00000769,cg00000807,cg00000884,...,ch.9.93402636R,ch.9.941347R,ch.9.945770F,ch.9.97139671F,ch.9.98936572R,ch.9.98937537R,ch.9.98957343R,ch.9.98959675F,ch.9.98989607R,ch.9.991104F
GSM989827,0.464197,0.941091,0.132014,0.686521,0.228244,0.177981,0.921818,0.061099,0.825423,0.887293,...,0.025619,0.043778,0.022659,0.061222,0.133692,0.042808,0.052589,0.035624,0.028066,0.043850
GSM989828,0.454883,0.939033,0.206917,0.619084,0.310879,0.144454,0.907529,0.066413,0.794975,0.874965,...,0.029295,0.038686,0.005095,0.052640,0.125270,0.036811,0.053343,0.075618,0.017428,0.032950
GSM989829,0.485764,0.918802,0.162861,0.635678,0.263215,0.185125,0.916278,0.062418,0.801009,0.861004,...,0.031233,0.035164,0.021444,0.058888,0.139105,0.042844,0.045973,0.126421,0.021752,0.022375
GSM989830,0.480785,0.929908,0.197780,0.610864,0.316761,0.162875,0.913187,0.069014,0.800715,0.868666,...,0.035059,0.042269,0.028587,0.056279,0.140601,0.042258,0.048733,0.084051,0.027504,0.053007
GSM989831,0.501219,0.934548,0.148437,0.651262,0.338289,0.198095,0.924478,0.063317,0.798748,0.850316,...,0.026308,0.037813,0.018626,0.057568,0.129993,0.039613,0.039254,0.165874,0.020889,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM990623,0.527496,0.958173,0.223068,0.709248,0.354296,0.203130,0.946202,0.046074,0.823633,0.865825,...,0.023085,0.048237,0.016319,0.072076,0.159212,0.049571,0.033060,0.093414,0.013654,0.014088
GSM990624,0.588331,0.982450,0.162180,0.535831,0.295598,0.228045,0.943319,0.052602,0.824448,0.903767,...,0.029672,0.011125,0.023837,0.058466,0.133899,0.055904,0.062442,0.101100,0.014193,0.000000
GSM990625,0.362994,0.954392,0.196430,0.664184,0.395724,0.205631,0.939465,0.060160,0.800421,0.871372,...,0.031308,0.056192,0.012054,0.053499,0.149589,0.048526,0.041087,0.049857,0.016840,0.041415
GSM990626,0.499145,0.931691,0.167477,0.665792,0.338117,0.234228,0.959109,0.042737,0.820515,0.918959,...,0.045281,0.040122,0.009878,0.060335,0.168269,0.056429,0.047835,0.028896,0.025346,0.052959


In [198]:
#ensure that all clock CpGs remain in the new set
common = intersection(hannum_test_40.columns.to_list(), sig_cpgs_original["CpG"].to_list())

if not sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    raise ValueError('Some Significant CpGs lost!')

In [199]:
len(common)

847

In [200]:
#Split the dataset into training and test subsets 

methyl_raw_train, methyl_raw_test, age_train, age_test = train_test_split(hannum_test_40, ages, test_size=0.2, random_state=42)


In [201]:
#Scale our data such that the fit is to the training set
scaler = StandardScaler().fit(methyl_raw_train)
methyl_train = scaler.transform(methyl_raw_train)

methyl_test = scaler.transform(methyl_raw_test)

In [202]:
#Read in age training data as a list
Y_train=age_train.values.ravel()

In [203]:
#Create Elastic Net model
elastic_netCV_40_i5000 = ElasticNetCV(l1_ratio = 0.5, n_alphas = 50, cv = 10, n_jobs=3, random_state = 42,
                             max_iter=5000, tol = 0.001, selection='cyclic')

In [204]:
#Train the model. 
elastic_netCV_40_i5000.fit(methyl_train,Y_train)

ElasticNetCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
             l1_ratio=0.5, max_iter=5000, n_alphas=50, n_jobs=3,
             normalize=False, positive=False, precompute='auto',
             random_state=42, selection='cyclic', tol=0.001, verbose=0)

In [205]:
#Save the Elastic Net model
dump(elastic_netCV_40_i5000, parent + '/Trained_Models/elastic_netCV_Hannum_40_i5000.joblib')

['D:\\elastic_netCV_Hannum_40_i5000.joblib']

In [206]:
#Save the Dataset
hannum_test_40.to_pickle(parent + '/MethylAndAges/hannum_40% nonsig removed.pkl')

# Find significant CpGs and compare
Here, as an assurance that our trainings are working, we will find the new Clock CpGs and compare them to the original

In [209]:
#Get the non-zero coefficients to get the significant cpgs. Runtime ~3 min
coeffs_40_i5000 = pd.DataFrame(elastic_netCV_40_i5000.coef_)
coeffs_40_i5000 = coeffs_40_i5000[(coeffs_40_i5000.T != 0).any()]
coeffs_40_i5000

,0
50,-0.001783
596,0.002247
653,-0.015071
760,-0.056703
804,0.026856
...,...
282719,-0.146395
283166,0.043709
283662,0.041517
283762,-0.000463


In [210]:
#Get significant CpGs and their indices
colnames_40_i5000 = pd.DataFrame(hannum_test_40.columns)
sig_cpgs_40_i5000 = colnames_40_i5000.iloc[coeffs_40_i5000.index]
sig_cpgs_40_i5000

,CpG
50,cg00003722
596,cg00042478
653,cg00047050
760,cg00055986
804,cg00058879
...,...
282719,ch.11.315572F
283166,ch.19.1716179R
283662,ch.4.60646130F
283762,ch.5.67026991F


In [211]:
#Compare original and new Clock CpGs
common = intersection(sig_cpgs_40_i5000["CpG"].to_list(), sig_cpgs_original["CpG"].to_list())

if  sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    print('NO Significant CpGs lost!')

In [212]:
len(common)

704

In [213]:
#Find the original Clock CpGs, which are not clock CpGs in the new model
loss = [value for value in sig_cpgs_original["CpG"].to_list() if value not in sig_cpgs_40_i5000["CpG"].to_list()]
len(loss)

143

# Now remove 50% CpGs
Here we will remove 50% of nonsig CpGs, create the dataset, and train an EN model

In [214]:
#Find number to be removed
num_removed = int(len_nonsig_original*0.50)

In [215]:
#Generate CpGs to be removed
list_removed = random.sample(range(0, len_nonsig_original), num_removed)

cpgs_removed_50 = nonsig_cpgs_original.iloc[list_removed]
cpgs_removed_50

,CpG
"(cg09023743,)",cg09023743
"(cg24135583,)",cg24135583
"(cg00147095,)",cg00147095
"(cg05766136,)",cg05766136
"(cg07024339,)",cg07024339
...,...
"(cg27357918,)",cg27357918
"(cg12262564,)",cg12262564
"(cg10474881,)",cg10474881
"(cg24425531,)",cg24425531


In [216]:
#Generate a new data set with the randomly selected CpGs removed
hannum_test_50 = hannum_test.drop(cpgs_removed_50["CpG"].to_list(), axis=1)

hannum_test_50

CpG,cg00000029,cg00000108,cg00000165,cg00000236,cg00000289,cg00000321,cg00000622,cg00000769,cg00000905,cg00000957,...,ch.9.84051654F,ch.9.84078312F,ch.9.90287778F,ch.9.90621653R,ch.9.93402636R,ch.9.96055087R,ch.9.98936572R,ch.9.98937537R,ch.9.98959675F,ch.9.98989607R
GSM989827,0.464197,0.941091,0.132014,0.717861,0.686521,0.228244,0.016508,0.061099,0.080157,0.868008,...,0.027931,0.025627,0.017504,0.026138,0.025619,0.109918,0.133692,0.042808,0.035624,0.028066
GSM989828,0.454883,0.939033,0.206917,0.723935,0.619084,0.310879,0.005747,0.066413,0.077787,0.875176,...,0.016973,0.023624,0.019708,0.028303,0.029295,0.076996,0.125270,0.036811,0.075618,0.017428
GSM989829,0.485764,0.918802,0.162861,0.719196,0.635678,0.263215,0.012197,0.062418,0.081269,0.882586,...,0.020021,0.043318,0.018805,0.032412,0.031233,0.070694,0.139105,0.042844,0.126421,0.021752
GSM989830,0.480785,0.929908,0.197780,0.704061,0.610864,0.316761,0.019945,0.069014,0.079151,0.872400,...,0.021958,0.029685,0.026202,0.027365,0.035059,0.094749,0.140601,0.042258,0.084051,0.027504
GSM989831,0.501219,0.934548,0.148437,0.754913,0.651262,0.338289,0.000000,0.063317,0.091947,0.883432,...,0.016914,0.025134,0.031734,0.022399,0.026308,0.110543,0.129993,0.039613,0.165874,0.020889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM990623,0.527496,0.958173,0.223068,0.778959,0.709248,0.354296,0.001778,0.046074,0.092683,0.900546,...,0.023569,0.019155,0.004092,0.030897,0.023085,0.079741,0.159212,0.049571,0.093414,0.013654
GSM990624,0.588331,0.982450,0.162180,0.796869,0.535831,0.295598,0.006685,0.052602,0.077472,0.903790,...,0.018592,0.019228,0.038536,0.030161,0.029672,0.117762,0.133899,0.055904,0.101100,0.014193
GSM990625,0.362994,0.954392,0.196430,0.713020,0.664184,0.395724,0.003822,0.060160,0.094486,0.894170,...,0.048834,0.008852,0.033379,0.034526,0.031308,0.079249,0.149589,0.048526,0.049857,0.016840
GSM990626,0.499145,0.931691,0.167477,0.730215,0.665792,0.338117,0.000000,0.042737,0.069827,0.900420,...,0.038441,0.037897,0.034949,0.023454,0.045281,0.090751,0.168269,0.056429,0.028896,0.025346


In [217]:
#ensure that all clock CpGs remain in the new set
common = intersection(hannum_test_50.columns.to_list(), sig_cpgs_original["CpG"].to_list())

if not sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    raise ValueError('Some Significant CpGs lost!')

In [218]:
len(common)

847

In [219]:
#Split the dataset into training and test subsets 

methyl_raw_train, methyl_raw_test, age_train, age_test = train_test_split(hannum_test_50, ages, test_size=0.2, random_state=42)


In [220]:
#Scale our data such that the fit is to the training set
scaler = StandardScaler().fit(methyl_raw_train)
methyl_train = scaler.transform(methyl_raw_train)

methyl_test = scaler.transform(methyl_raw_test)

In [221]:
#Read in age training data as a list
Y_train=age_train.values.ravel()

In [222]:
#Create Elastic Net model
elastic_netCV_50_i5000 = ElasticNetCV(l1_ratio = 0.5, n_alphas = 50, cv = 10, n_jobs=3, random_state = 42,
                             max_iter=5000, tol = 0.001, selection='cyclic')

In [223]:
#Train the model. 
elastic_netCV_50_i5000.fit(methyl_train,Y_train)

ElasticNetCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
             l1_ratio=0.5, max_iter=5000, n_alphas=50, n_jobs=3,
             normalize=False, positive=False, precompute='auto',
             random_state=42, selection='cyclic', tol=0.001, verbose=0)

In [224]:
#Save the Elastic Net model
dump(elastic_netCV_50_i5000, parent + '/Trained_Models/elastic_netCV_Hannum_50_i5000.joblib')

['D:\\elastic_netCV_Hannum_50_i5000.joblib']

In [225]:
#Save the Dataset
hannum_test_50.to_pickle(parent + '/MethylAndAges/hannum_50% nonsig removed.pkl')

# Find significant CpGs and compare
Here, as an assurance that our trainings are working, we will find the new Clock CpGs and compare them to the original

In [226]:
#Get the non-zero coefficients to get the significant cpgs. Runtime ~3 min
coeffs_50_i5000 = pd.DataFrame(elastic_netCV_50_i5000.coef_)
coeffs_50_i5000 = coeffs_50_i5000[(coeffs_50_i5000.T != 0).any()]
coeffs_50_i5000

,0
34,-0.017769
509,0.007456
551,-0.023993
643,-0.061178
661,0.028755
...,...
235464,-0.016778
235775,-0.143726
236161,0.049972
236566,0.037090


In [227]:
#Get significant CpGs and their indices
colnames_50_i5000 = pd.DataFrame(hannum_test_50.columns)
sig_cpgs_50_i5000 = colnames_50_i5000.iloc[coeffs_50_i5000.index]
sig_cpgs_50_i5000

,CpG
34,cg00003722
509,cg00042478
551,cg00047050
643,cg00055986
661,cg00058879
...,...
235464,cg27662246
235775,ch.11.315572F
236161,ch.19.1716179R
236566,ch.4.60646130F


In [228]:
#Compare original and new Clock CpGs
common = intersection(sig_cpgs_50_i5000["CpG"].to_list(), sig_cpgs_original["CpG"].to_list())

if  sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    print('NO Significant CpGs lost!')

In [229]:
len(common)

706

In [230]:
#Find the original Clock CpGs, which are not clock CpGs in the new model
loss = [value for value in sig_cpgs_original["CpG"].to_list() if value not in sig_cpgs_50_i5000["CpG"].to_list()]
len(loss)

141

# Now remove 60% CpGs
Here we will remove 60% of nonsig CpGs, create the dataset, and train an EN model

In [231]:
#Find number to be removed
num_removed = int(len_nonsig_original*0.60)

In [232]:
#Generate CpGs to be removed
list_removed = random.sample(range(0, len_nonsig_original), num_removed)

cpgs_removed_60 = nonsig_cpgs_original.iloc[list_removed]
cpgs_removed_60

,CpG
"(cg07333350,)",cg07333350
"(cg16595365,)",cg16595365
"(cg16745033,)",cg16745033
"(cg03880841,)",cg03880841
"(cg04566534,)",cg04566534
...,...
"(cg00014085,)",cg00014085
"(cg00993651,)",cg00993651
"(cg21005948,)",cg21005948
"(cg09145147,)",cg09145147


In [233]:
#Generate a new data set with the randomly selected CpGs removed
hannum_test_60 = hannum_test.drop(cpgs_removed_60["CpG"].to_list(), axis=1)

hannum_test_60

CpG,cg00000236,cg00000289,cg00000363,cg00000714,cg00000807,cg00000948,cg00001099,cg00001510,cg00001582,cg00001594,...,ch.9.77067993R,ch.9.7776528F,ch.9.79515146F,ch.9.84366407R,ch.9.87682774F,ch.9.898515R,ch.9.919537F,ch.9.93402636R,ch.9.945770F,ch.9.991104F
GSM989827,0.717861,0.686521,0.338483,0.177981,0.825423,0.860963,0.724557,0.515920,0.059022,0.014163,...,0.046605,0.022716,0.040809,0.012241,0.114574,0.022865,0.092026,0.025619,0.022659,0.043850
GSM989828,0.723935,0.619084,0.418998,0.144454,0.794975,0.843149,0.773727,0.477897,0.076662,0.014231,...,0.039543,0.022132,0.051790,0.012178,0.105732,0.024127,0.114144,0.029295,0.005095,0.032950
GSM989829,0.719196,0.635678,0.424736,0.185125,0.801009,0.868810,0.800382,0.502154,0.071017,0.014148,...,0.048583,0.034782,0.049825,0.010776,0.097549,0.028466,0.110516,0.031233,0.021444,0.022375
GSM989830,0.704061,0.610864,0.398772,0.162875,0.800715,0.864800,0.836739,0.499023,0.071929,0.026563,...,0.061536,0.025360,0.049826,0.010396,0.111340,0.029368,0.102213,0.035059,0.028587,0.053007
GSM989831,0.754913,0.651262,0.366965,0.198095,0.798748,0.861199,0.783540,0.482960,0.075170,0.021058,...,0.044261,0.039565,0.050379,0.019928,0.131276,0.038676,0.111282,0.026308,0.018626,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM990623,0.778959,0.709248,0.396241,0.203130,0.823633,0.855629,0.696408,0.513312,0.089451,0.000000,...,0.061952,0.040135,0.063035,0.022770,0.136908,0.019815,0.120232,0.023085,0.016319,0.014088
GSM990624,0.796869,0.535831,0.301319,0.228045,0.824448,0.851356,0.781041,0.502886,0.099329,0.000000,...,0.068033,0.023464,0.056981,0.000000,0.122025,0.050422,0.121977,0.029672,0.023837,0.000000
GSM990625,0.713020,0.664184,0.445179,0.205631,0.800421,0.873054,0.622539,0.468815,0.096933,0.009019,...,0.044174,0.039434,0.031938,0.029021,0.109536,0.029941,0.147659,0.031308,0.012054,0.041415
GSM990626,0.730215,0.665792,0.383953,0.234228,0.820515,0.862613,0.727681,0.555657,0.074847,0.000000,...,0.046622,0.036520,0.060896,0.042369,0.110954,0.027562,0.145616,0.045281,0.009878,0.052959


In [234]:
#ensure that all clock CpGs remain in the new set
common = intersection(hannum_test_60.columns.to_list(), sig_cpgs_original["CpG"].to_list())

if not sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    raise ValueError('Some Significant CpGs lost!')

In [235]:
len(common)

847

In [236]:
#Split the dataset into training and test subsets 

methyl_raw_train, methyl_raw_test, age_train, age_test = train_test_split(hannum_test_60, ages, test_size=0.2, random_state=42)


In [237]:
#Scale our data such that the fit is to the training set
scaler = StandardScaler().fit(methyl_raw_train)
methyl_train = scaler.transform(methyl_raw_train)

methyl_test = scaler.transform(methyl_raw_test)

In [238]:
#Read in age training data as a list
Y_train=age_train.values.ravel()

In [239]:
#Create Elastic Net model
elastic_netCV_60_i5000 = ElasticNetCV(l1_ratio = 0.5, n_alphas = 50, cv = 10, n_jobs=3, random_state = 42,
                             max_iter=5000, tol = 0.001, selection='cyclic')

In [240]:
#Train the model. 
elastic_netCV_60_i5000.fit(methyl_train,Y_train)

ElasticNetCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
             l1_ratio=0.5, max_iter=5000, n_alphas=50, n_jobs=3,
             normalize=False, positive=False, precompute='auto',
             random_state=42, selection='cyclic', tol=0.001, verbose=0)

In [241]:
#Save the Elastic Net model
dump(elastic_netCV_60_i5000, parent + '/Trained_Models/elastic_netCV_Hannum_60_i5000.joblib')

['D:\\elastic_netCV_Hannum_60_i5000.joblib']

In [242]:
#Save the Dataset
hannum_test_60.to_pickle(parent + '/MethylAndAges/hannum_60% nonsig removed.pkl')

# Now remove 70% CpGs
Here we will remove 70% of nonsig CpGs, create the dataset, and train an EN model

In [243]:
#Find number to be removed
num_removed = int(len_nonsig_original*0.70)

In [244]:
#Generate CpGs to be removed
list_removed = random.sample(range(0, len_nonsig_original), num_removed)

cpgs_removed_70 = nonsig_cpgs_original.iloc[list_removed]
cpgs_removed_70

,CpG
"(cg06446456,)",cg06446456
"(cg04442892,)",cg04442892
"(cg22939709,)",cg22939709
"(cg16937468,)",cg16937468
"(cg17883035,)",cg17883035
...,...
"(cg23404435,)",cg23404435
"(cg07703654,)",cg07703654
"(cg13867630,)",cg13867630
"(cg07818646,)",cg07818646


In [245]:
#Generate a new data set with the randomly selected CpGs removed
hannum_test_70 = hannum_test.drop(cpgs_removed_70["CpG"].to_list(), axis=1)

hannum_test_70

CpG,cg00000108,cg00000109,cg00000165,cg00000658,cg00000721,cg00000807,cg00001261,cg00001349,cg00001510,cg00001534,...,ch.9.75018133F,ch.9.80193246F,ch.9.82095949F,ch.9.83519450F,ch.9.84051654F,ch.9.88862796F,ch.9.96055087R,ch.9.97139671F,ch.9.98936572R,ch.9.991104F
GSM989827,0.941091,0.911182,0.132014,0.810140,0.921818,0.825423,0.468196,0.797852,0.515920,0.913358,...,0.027904,0.061157,0.000000,0.027217,0.027931,0.033558,0.109918,0.061222,0.133692,0.043850
GSM989828,0.939033,0.596455,0.206917,0.778277,0.907529,0.794975,0.480313,0.674899,0.477897,0.919409,...,0.035707,0.052380,0.014176,0.026020,0.016973,0.042730,0.076996,0.052640,0.125270,0.032950
GSM989829,0.918802,0.870333,0.162861,0.768844,0.916278,0.801009,0.468173,0.820352,0.502154,0.907879,...,0.033678,0.046989,0.010377,0.029140,0.020021,0.038133,0.070694,0.058888,0.139105,0.022375
GSM989830,0.929908,0.889689,0.197780,0.825187,0.913187,0.800715,0.457894,0.848526,0.499023,0.933358,...,0.048607,0.051310,0.007367,0.026216,0.021958,0.041635,0.094749,0.056279,0.140601,0.053007
GSM989831,0.934548,0.890450,0.148437,0.816176,0.924478,0.798748,0.479263,0.859865,0.482960,0.938526,...,0.034880,0.066897,0.019311,0.020241,0.016914,0.031274,0.110543,0.057568,0.129993,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM990623,0.958173,0.922034,0.223068,0.825719,0.946202,0.823633,0.469275,0.772518,0.513312,0.968167,...,0.042167,0.071415,0.026261,0.009349,0.023569,0.035479,0.079741,0.072076,0.159212,0.014088
GSM990624,0.982450,0.855145,0.162180,0.821282,0.943319,0.824448,0.485354,0.782007,0.502886,0.967276,...,0.043346,0.038745,0.000000,0.015126,0.018592,0.045863,0.117762,0.058466,0.133899,0.000000
GSM990625,0.954392,0.927183,0.196430,0.810389,0.939465,0.800421,0.505709,0.811156,0.468815,0.943799,...,0.049650,0.045487,0.008841,0.013102,0.048834,0.055299,0.079249,0.053499,0.149589,0.041415
GSM990626,0.931691,0.900938,0.167477,0.864885,0.959109,0.820515,0.483767,0.627299,0.555657,0.942947,...,0.047479,0.047293,0.006198,0.012739,0.038441,0.036660,0.090751,0.060335,0.168269,0.052959


In [246]:
#ensure that all clock CpGs remain in the new set
common = intersection(hannum_test_70.columns.to_list(), sig_cpgs_original["CpG"].to_list())

if not sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    raise ValueError('Some Significant CpGs lost!')

In [247]:
len(common)

847

In [248]:
#Split the dataset into training and test subsets 

methyl_raw_train, methyl_raw_test, age_train, age_test = train_test_split(hannum_test_70, ages, test_size=0.2, random_state=42)


In [249]:
#Scale our data such that the fit is to the training set
scaler = StandardScaler().fit(methyl_raw_train)
methyl_train = scaler.transform(methyl_raw_train)

methyl_test = scaler.transform(methyl_raw_test)

In [250]:
#Read in age training data as a list
Y_train=age_train.values.ravel()

In [251]:
#Create Elastic Net model
elastic_netCV_70_i5000 = ElasticNetCV(l1_ratio = 0.5, n_alphas = 50, cv = 10, n_jobs=3, random_state = 42,
                             max_iter=5000, tol = 0.001, selection='cyclic')

In [252]:
#Train the model. 
elastic_netCV_70_i5000.fit(methyl_train,Y_train)

ElasticNetCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
             l1_ratio=0.5, max_iter=5000, n_alphas=50, n_jobs=3,
             normalize=False, positive=False, precompute='auto',
             random_state=42, selection='cyclic', tol=0.001, verbose=0)

In [253]:
#Save the Elastic Net model
dump(elastic_netCV_70_i5000, parent + '/Trained_Models/elastic_netCV_Hannum_70_i5000.joblib')

['D:\\elastic_netCV_Hannum_70_i5000.joblib']

In [254]:
#Save the Dataset
hannum_test_70.to_pickle(parent + '/MethylAndAges/hannum_70% nonsig removed.pkl')

# Now remove 80% CpGs
Here we will remove 80% of nonsig CpGs, create the dataset, and train an EN model

In [255]:
#Find number to be removed
num_removed = int(len_nonsig_original*0.80)

In [256]:
#Generate CpGs to be removed
list_removed = random.sample(range(0, len_nonsig_original), num_removed)

cpgs_removed_80 = nonsig_cpgs_original.iloc[list_removed]
cpgs_removed_80

,CpG
"(cg27548450,)",cg27548450
"(cg19284368,)",cg19284368
"(cg00627998,)",cg00627998
"(cg12927293,)",cg12927293
"(cg02714882,)",cg02714882
...,...
"(cg10408178,)",cg10408178
"(ch.1.64660926R,)",ch.1.64660926R
"(cg19268452,)",cg19268452
"(cg06933824,)",cg06933824


In [257]:
#Generate a new data set with the randomly selected CpGs removed
hannum_test_80 = hannum_test.drop(cpgs_removed_80["CpG"].to_list(), axis=1)

hannum_test_80

CpG,cg00000289,cg00000363,cg00000714,cg00000769,cg00001245,cg00001249,cg00001446,cg00001593,cg00001747,cg00001791,...,ch.9.22352682F,ch.9.22677811R,ch.9.2503392R,ch.9.26162534R,ch.9.319861R,ch.9.357218F,ch.9.592503R,ch.9.84051654F,ch.9.898515R,ch.9.90287778F
GSM989827,0.686521,0.338483,0.177981,0.061099,0.007187,0.884557,0.841705,0.940081,0.032287,0.879292,...,0.083510,0.045244,0.043881,0.033666,0.129984,0.018551,0.054992,0.027931,0.022865,0.017504
GSM989828,0.619084,0.418998,0.144454,0.066413,0.014777,0.870456,0.848453,0.940844,0.039957,0.899570,...,0.089932,0.058460,0.068208,0.030265,0.148534,0.024273,0.051482,0.016973,0.024127,0.019708
GSM989829,0.635678,0.424736,0.185125,0.062418,0.012149,0.828569,0.827013,0.908645,0.071048,0.902628,...,0.079791,0.052719,0.065189,0.027286,0.170413,0.027559,0.048902,0.020021,0.028466,0.018805
GSM989830,0.610864,0.398772,0.162875,0.069014,0.007205,0.844759,0.848894,0.902501,0.042836,0.897485,...,0.089994,0.045794,0.041330,0.042432,0.167148,0.024558,0.044365,0.021958,0.029368,0.026202
GSM989831,0.651262,0.366965,0.198095,0.063317,0.011335,0.884540,0.851069,0.923369,0.070497,0.899383,...,0.077181,0.054733,0.008419,0.039659,0.144483,0.036839,0.051306,0.016914,0.038676,0.031734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM990623,0.709248,0.396241,0.203130,0.046074,0.000000,0.870396,0.890067,0.920037,0.062286,0.892051,...,0.082117,0.053660,0.041275,0.040458,0.142507,0.017059,0.045372,0.023569,0.019815,0.004092
GSM990624,0.535831,0.301319,0.228045,0.052602,0.010234,0.875512,0.881997,0.950467,0.044318,0.929633,...,0.084776,0.048713,0.057736,0.032699,0.172612,0.000000,0.047083,0.018592,0.050422,0.038536
GSM990625,0.664184,0.445179,0.205631,0.060160,0.002174,0.902469,0.884437,0.908135,0.068729,0.845768,...,0.072197,0.066774,0.078584,0.034764,0.182909,0.014168,0.064582,0.048834,0.029941,0.033379
GSM990626,0.665792,0.383953,0.234228,0.042737,0.000000,0.835348,0.878947,0.901941,0.059289,0.887869,...,0.093182,0.056161,0.045836,0.030440,0.178997,0.037096,0.066491,0.038441,0.027562,0.034949


In [258]:
#ensure that all clock CpGs remain in the new set
common = intersection(hannum_test_80.columns.to_list(), sig_cpgs_original["CpG"].to_list())

if not sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    raise ValueError('Some Significant CpGs lost!')

In [259]:
len(common)

847

In [260]:
#Split the dataset into training and test subsets 

methyl_raw_train, methyl_raw_test, age_train, age_test = train_test_split(hannum_test_80, ages, test_size=0.2, random_state=42)


In [261]:
#Scale our data such that the fit is to the training set
scaler = StandardScaler().fit(methyl_raw_train)
methyl_train = scaler.transform(methyl_raw_train)

methyl_test = scaler.transform(methyl_raw_test)

In [262]:
#Read in age training data as a list
Y_train=age_train.values.ravel()

In [263]:
#Create Elastic Net model
elastic_netCV_80_i5000 = ElasticNetCV(l1_ratio = 0.5, n_alphas = 50, cv = 10, n_jobs=3, random_state = 42,
                             max_iter=5000, tol = 0.001, selection='cyclic')

In [264]:
#Train the model. 
elastic_netCV_80_i5000.fit(methyl_train,Y_train)

ElasticNetCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
             l1_ratio=0.5, max_iter=5000, n_alphas=50, n_jobs=3,
             normalize=False, positive=False, precompute='auto',
             random_state=42, selection='cyclic', tol=0.001, verbose=0)

In [265]:
#Save the Elastic Net model
dump(elastic_netCV_80_i5000, parent + '/Trained_Models/elastic_netCV_Hannum_80_i5000.joblib')

['D:\\elastic_netCV_Hannum_80_i5000.joblib']

In [266]:
#Save the Dataset
hannum_test_80.to_pickle(parent + '/MethylAndAges/hannum_80% nonsig removed.pkl')

# Now remove 90% CpGs
Here we will remove 90% of nonsig CpGs, create the dataset, and train an EN model

In [267]:
#Find number to be removed
num_removed = int(len_nonsig_original*0.90)

In [268]:
#Generate CpGs to be removed
list_removed = random.sample(range(0, len_nonsig_original), num_removed)

cpgs_removed_90 = nonsig_cpgs_original.iloc[list_removed]
cpgs_removed_90

,CpG
"(cg13554773,)",cg13554773
"(cg24375244,)",cg24375244
"(cg08490233,)",cg08490233
"(cg18449734,)",cg18449734
"(cg24670330,)",cg24670330
...,...
"(cg00071565,)",cg00071565
"(cg14747065,)",cg14747065
"(cg15624376,)",cg15624376
"(cg24321113,)",cg24321113


In [269]:
#Generate a new data set with the randomly selected CpGs removed
hannum_test_90 = hannum_test.drop(cpgs_removed_90["CpG"].to_list(), axis=1)

hannum_test_90

CpG,cg00000029,cg00000165,cg00000948,cg00001874,cg00002116,cg00002190,cg00002719,cg00002808,cg00003305,cg00004061,...,ch.9.1872490F,ch.9.1874451R,ch.9.2262725R,ch.9.25704165R,ch.9.357218F,ch.9.38721634R,ch.9.592503R,ch.9.75018133F,ch.9.80193246F,ch.9.837340R
GSM989827,0.464197,0.132014,0.860963,0.511644,0.006924,0.716471,0.000840,0.828079,0.168100,0.741166,...,0.062537,0.061076,0.034438,0.045292,0.018551,0.040409,0.054992,0.027904,0.061157,0.030293
GSM989828,0.454883,0.206917,0.843149,0.440047,0.011229,0.709477,0.014389,0.805960,0.125855,0.732815,...,0.060245,0.063350,0.029644,0.022998,0.024273,0.050594,0.051482,0.035707,0.052380,0.029746
GSM989829,0.485764,0.162861,0.868810,0.512959,0.007719,0.686167,0.000862,0.862125,0.132346,0.724785,...,0.062783,0.047575,0.031018,0.019265,0.027559,0.049915,0.048902,0.033678,0.046989,0.032652
GSM989830,0.480785,0.197780,0.864800,0.460729,0.012096,0.688517,0.015819,0.827201,0.135731,0.701649,...,0.067479,0.022312,0.030727,0.031012,0.024558,0.048465,0.044365,0.048607,0.051310,0.032679
GSM989831,0.501219,0.148437,0.861199,0.591874,0.005136,0.749695,0.015379,0.859516,0.160379,0.691237,...,0.065556,0.046019,0.028703,0.010877,0.036839,0.049216,0.051306,0.034880,0.066897,0.040077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM990623,0.527496,0.223068,0.855629,0.578767,0.000000,0.711831,0.000000,0.854859,0.177433,0.743738,...,0.054810,0.000000,0.042652,0.067631,0.017059,0.051652,0.045372,0.042167,0.071415,0.033501
GSM990624,0.588331,0.162180,0.851356,0.542855,0.000000,0.781664,0.000000,0.894122,0.091437,0.608635,...,0.051771,0.050872,0.047596,0.007682,0.000000,0.054790,0.047083,0.043346,0.038745,0.024919
GSM990625,0.362994,0.196430,0.873054,0.534792,0.000000,0.729962,0.015883,0.856993,0.130402,0.703335,...,0.074410,0.015236,0.031047,0.088797,0.014168,0.066484,0.064582,0.049650,0.045487,0.030581
GSM990626,0.499145,0.167477,0.862613,0.584563,0.002463,0.743060,0.002722,0.879425,0.131272,0.827657,...,0.086502,0.014753,0.054051,0.025598,0.037096,0.065505,0.066491,0.047479,0.047293,0.035774


In [270]:
#ensure that all clock CpGs remain in the new set
common = intersection(hannum_test_90.columns.to_list(), sig_cpgs_original["CpG"].to_list())

if not sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    raise ValueError('Some Significant CpGs lost!')

In [271]:
len(common)

847

In [272]:
#Split the dataset into training and test subsets 

methyl_raw_train, methyl_raw_test, age_train, age_test = train_test_split(hannum_test_90, ages, test_size=0.2, random_state=42)


In [273]:
#Scale our data such that the fit is to the training set
scaler = StandardScaler().fit(methyl_raw_train)
methyl_train = scaler.transform(methyl_raw_train)

methyl_test = scaler.transform(methyl_raw_test)

In [274]:
#Read in age training data as a list
Y_train=age_train.values.ravel()

In [275]:
#Create Elastic Net model
elastic_netCV_90_i5000 = ElasticNetCV(l1_ratio = 0.5, n_alphas = 50, cv = 10, n_jobs=3, random_state = 42,
                             max_iter=5000, tol = 0.001, selection='cyclic')

In [276]:
#Train the model. 
elastic_netCV_90_i5000.fit(methyl_train,Y_train)

ElasticNetCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
             l1_ratio=0.5, max_iter=5000, n_alphas=50, n_jobs=3,
             normalize=False, positive=False, precompute='auto',
             random_state=42, selection='cyclic', tol=0.001, verbose=0)

In [277]:
#Save the Elastic Net model
dump(elastic_netCV_90_i5000, parent + '/Trained_Models/elastic_netCV_Hannum_90_i5000.joblib')

['D:\\elastic_netCV_Hannum_90_i5000.joblib']

In [278]:
#Save the Dataset
hannum_test_90.to_pickle(parent + '/MethylAndAges/hannum_90% nonsig removed.pkl')

# Now remove 100% CpGs
Here we will remove 100% of nonsig CpGs, create the dataset, and train an EN model

In [279]:
#Find number to be removed
num_removed = int(len_nonsig_original*1.00)

In [280]:
#Generate CpGs to be removed
list_removed = random.sample(range(0, len_nonsig_original), num_removed)

cpgs_removed_100 = nonsig_cpgs_original.iloc[list_removed]
cpgs_removed_100

,CpG
"(cg01804370,)",cg01804370
"(cg12731092,)",cg12731092
"(cg20614157,)",cg20614157
"(cg04369837,)",cg04369837
"(cg10652351,)",cg10652351
...,...
"(cg11544882,)",cg11544882
"(cg08234618,)",cg08234618
"(cg17503200,)",cg17503200
"(cg12636435,)",cg12636435


In [281]:
#Generate a new data set with the randomly selected CpGs removed
hannum_test_100 = hannum_test.drop(cpgs_removed_100["CpG"].to_list(), axis=1)

hannum_test_100

CpG,cg00042478,cg00047050,cg00055986,cg00058879,cg00120464,cg00123072,cg00168942,cg00246121,cg00281129,cg00315391,...,cg27506082,cg27550325,cg27569300,cg27662246,ch.11.110310046R,ch.11.315572F,ch.19.1716179R,ch.4.60646130F,ch.6.2574971F,ch.6.33611621F
GSM989827,0.988354,0.155244,0.267208,0.180338,0.927025,0.877821,0.535961,0.334566,0.922840,0.055133,...,0.027434,0.698098,0.070001,0.878847,0.026769,0.202250,0.092257,0.026170,0.008833,0.041219
GSM989828,0.965200,0.113820,0.297573,0.174198,0.932776,0.924583,0.478570,0.321031,0.933906,0.035405,...,0.021614,0.622693,0.130124,0.862047,0.022496,0.048027,0.075512,0.041567,0.020213,0.040775
GSM989829,0.956607,0.132763,0.354057,0.175213,0.900056,0.912706,0.564775,0.421436,0.931744,0.026102,...,0.028287,0.719761,0.102406,0.887271,0.021725,0.128649,0.077525,0.037895,0.019721,0.072077
GSM989830,0.917381,0.143769,0.269180,0.132281,0.876115,0.890417,0.542968,0.391868,0.903954,0.036156,...,0.027812,0.553545,0.091364,0.883088,0.022068,0.103743,0.101558,0.028206,0.034047,0.035663
GSM989831,0.948417,0.151902,0.321703,0.204761,0.914204,0.869289,0.574587,0.422332,0.925779,0.049095,...,0.040365,0.625966,0.087164,0.871559,0.011955,0.258040,0.083336,0.036823,0.032085,0.052129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM990623,0.986848,0.140337,0.328201,0.264072,0.918653,0.942336,0.532538,0.431387,0.966526,0.032266,...,0.031269,0.784360,0.111821,0.888363,0.006809,0.083340,0.095607,0.024050,0.040195,0.058584
GSM990624,0.942068,0.099530,0.331531,0.239304,0.900308,0.979489,0.459146,0.410168,0.950184,0.026851,...,0.028993,0.786392,0.102681,0.904370,0.051089,0.084007,0.064258,0.034127,0.031378,0.041560
GSM990625,0.987954,0.104357,0.517700,0.174299,0.890416,0.869857,0.565868,0.373482,0.935472,0.035351,...,0.012868,0.614032,0.086842,0.890609,0.008158,0.030281,0.090461,0.021934,0.008924,0.061846
GSM990626,0.965633,0.132477,0.222999,0.234759,0.947308,0.904380,0.601958,0.409950,0.949358,0.021665,...,0.020079,0.615677,0.072486,0.901227,0.046431,0.019310,0.099772,0.030709,0.031022,0.072027


In [282]:
#ensure that all clock CpGs remain in the new set
common = intersection(hannum_test_100.columns.to_list(), sig_cpgs_original["CpG"].to_list())

if not sorted(common) == sorted(sig_cpgs_original["CpG"].to_list()):
    raise ValueError('Some Significant CpGs lost!')

In [283]:
len(common)

847

In [284]:
#Split the dataset into training and test subsets 

methyl_raw_train, methyl_raw_test, age_train, age_test = train_test_split(hannum_test_100, ages, test_size=0.2, random_state=42)


In [285]:
#Scale our data such that the fit is to the training set
scaler = StandardScaler().fit(methyl_raw_train)
methyl_train = scaler.transform(methyl_raw_train)

methyl_test = scaler.transform(methyl_raw_test)

In [286]:
#Read in age training data as a list
Y_train=age_train.values.ravel()

In [287]:
#Create Elastic Net model
elastic_netCV_100_i5000 = ElasticNetCV(l1_ratio = 0.5, n_alphas = 50, cv = 10, n_jobs=3, random_state = 42,
                             max_iter=5000, tol = 0.001, selection='cyclic')

In [288]:
#Train the model. 
elastic_netCV_100_i5000.fit(methyl_train,Y_train)

ElasticNetCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
             l1_ratio=0.5, max_iter=5000, n_alphas=50, n_jobs=3,
             normalize=False, positive=False, precompute='auto',
             random_state=42, selection='cyclic', tol=0.001, verbose=0)

In [289]:
#Save the Elastic Net model
dump(elastic_netCV_100_i5000, parent + '/Trained_Models/elastic_netCV_Hannum_100_i5000.joblib')

['D:\\elastic_netCV_Hannum_100_i5000.joblib']

In [290]:
#Save the Dataset
hannum_test_100.to_pickle(parent + '/MethylAndAges/hannum_100% nonsig removed.pkl')